# 0. **Import**

In [1]:
import imp  # Deprecated module, consider using importlib
import math  # Mathematical functions
import random  # Random number generation
import os  # Operating system interfaces
import re  # Regular expressions
import copy  # Object copying
import itertools  # Iterators for efficient looping
from collections import Counter  # Count hashable items
from functools import cmp_to_key  # Convert old-style comparisons to key functions
import imageio  # Image input/output
import networkx as nx  # Graph analysis
import csv  # CSV file reading and writing
import subprocess  # Subprocess management

from os.path import exists  # Check if a file exists
import pickle  # Object serialization
from os import listdir  # List directory contents
from os.path import isfile, join  # File path operations
import dill  # Extended pickle module

import numpy as np  # Numerical computing
from numpy import linalg as LA  # Linear algebra operations
from shapely.geometry import Polygon, Point  # Geometric objects

import matplotlib.pyplot as plt  # Plotting library
import matplotlib.colors as mcolors  # Color handling in matplotlib
import pandas as pd  # Data manipulation and analysis
import pyarrow.feather as feather  # Feather format for fast data storage

import manifold_util as mu  # Custom manifold utilities
import database  # Database interaction
import polygon_util as pu  # Polygon utilities
import icp  # Iterative closest point algorithm

import NEB_binded_main  # NEB-related module
import NEB_main  # NEB-related module

from lammps import PyLammps  # LAMMPS simulation package
import lammps_util.Nudged_Elastic_Band as NEB  # NEB utility for LAMMPS
import lammps_util.dynamic_file_template as dynamic_file_template  # Dynamic file templates for LAMMPS
import lammps_util.sim_param  # Simulation parameters for LAMMPS
import lammps_util.sim_param as sim_param  # Alias for convenience
import lammps_util.my_lammps as mylammps  # Custom LAMMPS interface
from lammps_util.util import Util  # Utility functions for LAMMPS
import lammps_util.logger as logger  # Logging utilities
import lammps_util.analysis as analysis  # Analysis tools for LAMMPS
import lammps_util.protein_template as protein_template  # Protein templates for LAMMPS
from lammps_util.dynamic_file_template import Save_file_full_sim  # Save file utility for LAMMPS simulations
import lammps_util.dump as dump  # Dump file handling
import simulation as simulation  # Simulation module
import internal_node_main as inm  # Internal node management

from sklearn.neighbors import NearestNeighbors  # Nearest neighbor search
from scipy.stats import qmc  # Quasi-Monte Carlo sampling
import statsmodels.stats.proportion as ssp  # Statistical proportion testing
from scipy.interpolate import griddata  # Interpolation
from sklearn.decomposition import PCA  # Principal Component Analysis

import dumpWorker  # Worker process for dump file handling
import multiprocessing  # Parallel processing utilities
import dill  # Extended serialization for multiprocessing
from multiprocessing import Pool  # Multiprocessing pool for parallel execution
import pandas  # Data analysis

from functools import partial

from matplotlib import pyplot, transforms
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as ticker
import matplotlib

/var/folders/vx/m0jnfzsj37zcnfbqpjdcj7000000gn/T/ipykernel_29151/560332721.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


<module 'NEB_binded_main' from '/Users/mathieuouellet/Dropbox/My Mac (mathieu’s MacBook Pro)/Desktop/AMP/AMP/src/polygon/NEB_binded_main.py'>

# 1. **External Polygons**

This section set up the database for the polygon and the simulation values.

Then random polygons are added 



In [ ]:


n=5
db = database.Database("data/prion_database")
db.set_up_polygon(5,[1.0]*5, 5*2*math.pi/360.0, True)
db.set_up_pairs( max_fitness_healthy=2.001, min_fitness_prion=3.0, distance_binded=0.08 , hp_pp_dif=0.2, n_point_max=5 )
db.set_up_internal_node(nb_node_in=2, max_nb_try=200, nb_solution_wanted=1,min_avg_distance_sol=0.1 )
db.set_up_interaction( mass=(1,1), K =(1,1) )
db.set_up_stability(n_test_temperature=6, start_temp=0.00001, max_temp=0.01, maxtime=50)




for i in range(100000):
    db.add_polygon(pu.find_random_poly(n, 0),{"fitness":None}, commit=False)
    if i %1000==0:
        db.commit_polygon()
db.commit_polygon()
db.lock_polygon()

In [ ]:
# Create a polygon with `n_create` vertices and attempt to find valid ones.
# Each polygon is generated up to `max_try_poly * n_batch` times, depending on 
# the probability of forming a valid polygon with a minimum angle of `min_angle_poly`.

n_create = 5
max_try_poly = 1000000
n_batch = 100
min_angle_poly = 10*2*math.pi/360.0
nb_try = 1000

for _ in range(nb_try):
    # Generate a set of valid conformer polygons
    sol_set_true = pu.find_set_of_conformer_polygon(
        [1]*n_create, max_try_poly, is_simple=True, min_angle=min_angle_poly)

    for sol in sol_set_true:
        poly1 = Polygon(sol)
        poly2 = Polygon(sol)

        # Estimate stacking using ICP with constraints on overlap and binding behavior
        try:
            fit_ordered, sol_ordered, output_index, intersect = pu.estimate_stacking_icp(
                poly1, poly2, n_point_max=n_create-1, avg_dist_tol=0.3, ratio_max_overlap_area=0.02, multiple_bind=False, output_index=True)
            print(fit_ordered[-1], sol)
        except:
            print("Error: Issue encountered during stacking estimation. Investigate further.")
            raise ValueError()

In [ ]:
# Plot a histogram of self-binding strenght


df = pandas.read_table("./data/data_n=5_self_binding_10_deg", delimiter=';')
columtn = df['fit']
print(len(columtn))
data = []
for i in range(len(columtn)):
    splitted = str(columtn[i]).split(" ")
    data.append(pu.rescale_fit(float(splitted[0])))

plt.rcParams["figure.figsize"] = (6, 4)
plt.hist(data, density=True, bins=30)
plt.ylabel('%')
plt.xlabel('binding ')
plt.yscale('log')
plt.savefig(f"figure/binding_side/distrib.svg")
plt.show()


In [ ]:
# Testing self-matching for n in range 2 to 4
# No solution is expected for n = 4

fit_obtained = 14.50885461236566 # Precomputed fitness value
length_obtained = [1, 1, 1, 1, 1, 1] # Edge lengths of the polygon
# Angles defining the polygon's shape
angles_obtained = (
    6.187885080269032, 0.724165427552586, 2.0304603419914993,
    3.849794768854707, 3.072941576789273, 5.179276910994469
)

# Compute the polygon's vertex positions
pos_obtained = pu.get_pos(length_obtained, (angles_obtained, (0, 0)))[:-1]
pos_obtained.append(pos_obtained[0]) # Close the polygon loop
poly1 = Polygon(pos_obtained)  # Create a Polygon object


# Check self-matching for n_to_check in [2, 3, 4]
for n_to_check in range(2, 5):

    # Attempt to find self-matching substructures within the polygon
    found_sol = pu.check_self_matching(
        n_to_check, poly1, avg_dist_tol=0.2, ratio_max_overlap_area=0.1)

    # Plot the original polygon and any found solutions
    for sol in found_sol:
        for poly in [poly1, sol]:
            x_p, y_p = poly.exterior.xy
            plt.xlim(-10, 10)
            plt.ylim(-10, 10)
            plt.plot(x_p, y_p)
            
    plt.show()

In [ ]:
# Print and plot random polygons that fit within the specified binding range

nb_to_print = 30  # Number of polygons to print
min_bind = 3.0  # Minimum binding threshold after rescaling
max_bind = 10000  # Maximum binding threshold after rescaling

# Load self-binding data from a tab-separated file
df = pandas.read_table("./data/data_n=5_self_binding_10_deg", delimiter=';')

column = df['fit']
list_w_cond = []  # List to store selected polygons that meet the binding condition

# Process each entry in the 'fit' column
for i in range(len(column)):
    list_part = re.split(r'\[|\]', column[i])  # Split to extract binding value and positions
    fit = float(list_part[0])  # Extract and convert the fitness value

    # Extract polygon vertex positions from the string
    list_pos_xy = re.split(r'\(|\)', list_part[1])
    position = [
        tuple(float(val.replace('\'', '')) for val in list_pos_xy[2 * i + 1].split(','))
        for i in range(len(list_pos_xy) // 2)
    ]

    # Check if the rescaled fit value falls within the specified range
    if min_bind < pu.rescale_fit(fit) < max_bind:
        polygon = Polygon(position)  # Create a polygon object
        list_w_cond.append((i, fit, polygon))

# Randomly select and process `nb_to_print` polygons
for i, fit, polygon in random.sample(list_w_cond, nb_to_print):

    # Estimate stacking using ICP with specific constraints
    fit_ordered, sol_ordered, output_index, intersect = pu.estimate_stacking_icp(
        polygon, polygon, n_point_max=4, avg_dist_tol=0.2,
        ratio_max_overlap_area=0.2, multiple_bind=True, output_index=True
    )

    # Print the polygon index and its fitness value
    print(i, fit)

    # Plot the original polygon and the transformed one from ICP
    sol = sol_ordered[-1]
    for poly in [polygon, sol]:
        x_p, y_p = poly.exterior.xy
        plt.xlim(-3, 3)
        plt.ylim(-3, 3)
        plt.plot(x_p, y_p)

    # Ensure equal aspect ratio and save the figure
    plt.gca().set_aspect('equal', adjustable='box')
    plt.savefig(f"figure/binding_side/{i}.svg")
    plt.show()

In [ ]:
# Load the full manifold for a given set of solutions (external polygon shape)
# This is purely for visualization purposes.


dim = 3  # Arbitrary choice for visualization; does not reflect actual physical dimensions

# Load the dataset containing all solutions
df_all = pandas.read_table("./data/data_n=5", delimiter=';')

# Extract fitness values and corresponding polygon positions
fit_list_all, pos_list_all = pu.read_txt_file_fit_pos(df_all)

# This manifold is only used for visualization, not for structural analysis.
full_manifold, full_manifold_pd = mu.learn_manifold(pos_list_all, dim=3)

In [ ]:
# Load and visualize self-binding solutions on the full manifold


# Load dataset of self-binding solutions
df_fitted = pandas.read_table("./data/data_n=5_self_binding_10_deg", delimiter=';')

# Extract fitness values and corresponding polygon positions
fit_list_fitted, pos_list_fitted = pu.read_txt_file_fit_pos(df_fitted)

angle_1,angle_2 = 60, 200

# Plot the full manifold without fitness values (purely structural)
fig, new_ax = mu.plot_manifold_3d(
    full_manifold_pd, fitness_list=None, angle_1=angle_1, angle_2=angle_2, alpha=0.000)

# Overlay self-binding solutions onto the manifold with full opacity
fig, new_ax = mu.plot_on_manifold_3d(pos_list_fitted, full_manifold, fig=fig, active_axis=new_ax,
                                     fitness_list=fit_list_fitted, angle_1=angle_1, angle_2=angle_2, alpha=1)

fig.savefig('/Users/mathieuouellet/Desktop/AMP/Figures/n_5_manifold_binding.png')

plt.show()

# 2. **Pair Selection for Prion Formation**

Now that we have tested some of the functions for generating polygons and checked whether they match, we can take the next step: **creating a database** and applying the same methods on a larger scale.  

By storing polygons in a structured database, we can systematically search for **healthy-prion pairs** and analyze their transformations more efficiently.  

To achieve this, we will use the script **`polygon_matching_main.py`**, which handles both polygon matching and database operations.  


The script  polygon_matching_main.py identifies and selects polygonal structures that could act as potential prion candidates based on their binding properties. It compares **healthy** and **prion** structures to find pairs that satisfy specific geometric and binding conditions. The goal is to determine whether a **healthy** structure can transition into a **prion** while maintaining its binding interactions.

1. **Load Data**  
   - Retrieve polygonal structures from the database.
   - Separate structures into *healthy* and *prion* candidates.
  
2. **Compute Binding Properties**  
   - Determine how strongly prions bind to themselves.
   - Evaluate how well healthy structures bind to prions.

3. **Filter Based on Selection Criteria**  
   A pair *(healthy, prion)* is considered valid if it meets the following conditions:

   - **COND1:** The healthy structure binds less than a maximum threshold.
   - **COND2:** The prion binds more than a minimum threshold.
   - **COND3:** The fitness of the *(healthy, prion)* binding is significantly lower than the *(prion, prion)* binding.
   - **COND4:** The fitness of the *(healthy, prion)* binding is at least as high as the *(healthy, healthy)* binding.
   - **COND5:** The healthy and prion structures must bind to the same sites on the prion.
   - **COND6:** The healthy structure must be able to morph into the prion while remaining bound.

4. **Store Valid Pairs in the Database**  
   - If a valid transition is found, the pair is recorded for further analysis.
   - Previously tested pairs are also stored to avoid redundant computations.

**Usage**
Run the script with:

```bash
python polygon_matching_main.py
```

The database path must be manually modified in the main section of the script:

```python
if __name__ == "__main__":
   db = database.Database("src/polygon/data/n=5_5deg_2")  # Change this path to your database location
   select_pairs(db)
```


In [ ]:
# we load the data base and set the parameter the next algorithm will use


db = database.Database("/Users/mathieuouellet/Desktop/n=5_5deg_2")

db.set_up_polygon(5, [1.0]*5, 5*2*math.pi/360.0, True)
db.set_up_pairs(max_fitness_healthy=2.001, min_fitness_prion=3.0,
                distance_binded=0.08, hp_pp_dif=0.2, n_point_max=5)
db.set_up_internal_node(nb_node_in=2, max_nb_try=200,
                        nb_solution_wanted=1, min_avg_distance_sol=0.1)
db.set_up_interaction(mass=(1, 1), K=(1, 1))
db.set_up_stability(n_test_temperature=6, start_temp=0.00001,
                    max_temp=0.01, maxtime=50)


# 3. **Internal nodes**



The `internal_node_main.py` script is responsible for generating, comparing, and validating internal node configurations for polygon-based structures. The script integrates MATLAB to optimize internal node placements while ensuring they meet connectivity constraints.

### **Key Functions**
#### **1. `start_matlab_engine()`**  

- Initializes the MATLAB engine and adds the required MATLAB scripts to the path.
- Returns an active MATLAB engine instance.

#### **2. `get_random_conn(n_out, nedge, npoint)`**  
- Generates a random connection between internal and external nodes while ensuring all nodes are connected.
- Uses random partitioning of edges to create connectivity groups.
- Ensures each node is part of a valid connection structure.
- Returns a list of connection groups.

#### **3. `compare_connection(connection1, connection2)`**  
- Compares two connection configurations to determine if they are equivalent.
- Uses permutations to check all possible mappings of internal nodes.
- Returns a boolean indicating equivalence and mapping information if they match.

#### **4. `compare_distance(inside_node_1, inside_node_2, perm)`**  
- Computes the average Euclidean distance between corresponding internal nodes.
- Used to check the similarity of two internal node configurations after mapping.
- Returns the average distance.


#### **5. `find_solution(db, eng, list_key_dict=None, nb_solution_wanted=None, max_nb_try=None, min_avg_distance_sol=None, nb_node_in=None, conn_func=None)`**
- Core function that iterates over database entries to find valid internal node configurations.
- Uses the MATLAB engine to optimize node placements.
- Ensures uniqueness of solutions before adding them to the database.
- Handles locking/unlocking of database entries to prevent conflicts in multi-threaded environments.
- Saves valid configurations to the database.

## Usage
The script can be executed directly, with user-defined parameters to control the search space. 




This cell counts the number of solutions stored in the database and identifies which external polygons successfully match with a prion partner. It tracks how these matches are stored and calculates the total number of solutions and the ratio of successful pairings.  

In [ ]:
#load solution found and count them 

total_sol, total_not_sol = 0, 0  
potential_key_pairs = []  # List to store key pairs with solutions
total_hp, total_hp_not = 0, 0  # Counters for healthy-prion pairs with and without solutions

# Iterate through all healthy keys in the database
for key_h in list(db.db_pairs):

    # Iterate through all prion keys associated with the healthy key
    for key_p in db.db_pairs[key_h].keys():

        # Check if there are solutions for this (healthy, prion) pair
        if 'sols' in db.db_pairs[key_h][key_p]:
            total_hp += 1  # Increment count of pairs with solutions
            potential_key_pairs.append((key_h, key_p))  # Store the key pair

            # Count the number of solutions for this pair
            total_sol += len(list(db.db_pairs[key_h][key_p]['sols']))
        else:
            total_hp_not += 1  # Increment count of pairs without solutions

# Print total number of healthy-prion pairs with solutions and their ratio to those without solutions
print(total_hp, total_hp / total_hp_not if total_hp_not != 0 else "N/A")  

# Print total number of solutions found
print(total_sol)  

# 4. **NEB Computation**


### **Nudged Elastic Band (NEB) Simulation for Polygon Transitions**

This script performs **Nudged Elastic Band (NEB) simulations** to analyze how a **healthy polygon** transitions into a **prion-like structure** under physical constraints. The goal is to compute **reaction pathways**, **binding energies**, and **morphological transformations** between these structures.

### **Key Functions**
#### **1. `create_executable()`**  
Generates an executable script for running **LAMMPS** simulations using MPI across multiple nodes.

#### **2. `_get_energy()`**  
Computes the energy of a given **protein configuration** based on bond stretching.

#### **3. `get_energy()`**  
Extracts NEB simulation data and calculates energy values for each intermediate structure.

#### **4. `get_NEB()`**  
Runs the **NEB simulation** for a given **(healthy, prion) pair** and stores the computed reaction coordinates and energy landscape.

#### **5. `neb_fill_database()`**  
Fills the database with **NEB results** by iterating through all healthy-prion pairs and storing computed **energy profiles** and **reaction coordinates**.

### **Workflow**
1. **Load the database** containing healthy and prion structures.
2. **Select polygon pairs** for which NEB simulations will be performed.
3. **Run LAMMPS simulations** to compute energy barriers and binding dynamics.
4. **Store results** in the database for further analysis.

### **Usage**
Run the script with:

```bash
python neb_simulation.py
```
Alternatively, you can specify a subset of polygon pairs to compute:



### **NEB_binded_main.py: Handling the Bonded Case**

The script **`NEB_binded_main.py`** operates similarly to **`neb_simulation.py`**, but it specifically handles cases where the **healthy and prion structures remain bonded during the transition**.  

It follows the same workflow—loading the database, selecting pairs, running **Nudged Elastic Band (NEB) simulations**, and storing results—but ensures that binding constraints are maintained throughout the process.  


In [ ]:
################################################################
#        load energy list and reaction coordinate H<->P        #
################################################################


nb_plotted = 0
rms_list = []
energy_list = []
temp_list = []
stab_list_h = []
stab_list_p = []
rc_coor_list = []
bind_coor_list = []
bind_energy_list = []
bind_pos_list = []
bind_angle_list = []

healthy_key = []
prion_key = []
sol_key = []

for key_h in list(db.db_pairs):

    for key_p in db.db_pairs[key_h].keys():
        if 'sols' in db.db_pairs[key_h][key_p]:
            for key_s in db.db_pairs[key_h][key_p]['sols'].keys():

                if type(db.db_pairs[key_h][key_p]['sols'][key_s]) != int:

                    reaction_coordinate = None
                    energy = None
                    RMS_hp = None
                    temperature_list = None
                    healthy_angle_list = None
                    prion_angle_list = None
                    binded_reaction_coordinate = None
                    binded_energy = None
                    binded_pos_list = None
                    binded_angle_list = None

                    if 'NEB_HP_energy' in db.db_pairs[key_h][key_p]['sols'][key_s]:
                        reaction_coordinate = db.db_pairs[key_h][key_p]['sols'][key_s]['NEB_HP_reaction_coordinate']
                        energy = db.db_pairs[key_h][key_p]['sols'][key_s]['NEB_HP_energy']

                    if 'RMS_hp' in db.db_pairs[key_h][key_p]['sols'][key_s]:
                        RMS_hp = db.db_pairs[key_h][key_p]['sols'][key_s]['RMS_hp']
                        temperature_list = db.db_pairs[key_h][key_p]['sols'][key_s]['RMS_stability_temp']
                        healthy_angle_list = [
                            x/RMS_hp for x in db.db_pairs[key_h][key_p]['sols'][key_s]['RMS_stability_healthy']]
                        prion_angle_list = [
                            x/RMS_hp for x in db.db_pairs[key_h][key_p]['sols'][key_s]['RMS_stability_prion']]

                    if 'NEB_HP_PP_energy' in db.db_pairs[key_h][key_p]['sols'][key_s]:

                        binded_reaction_coordinate = db.db_pairs[key_h][key_p][
                            'sols'][key_s]['NEB_HP_PP_reaction_coordinate']
                        binded_energy = db.db_pairs[key_h][key_p]['sols'][key_s]['NEB_HP_PP_energy']

                    if 'NEB_HP_PP_pos_list' in db.db_pairs[key_h][key_p]['sols'][key_s]:
                        binded_pos_list = db.db_pairs[key_h][key_p]['sols'][key_s]['NEB_HP_PP_pos_list']
                        binded_angle_list = db.db_pairs[key_h][key_p]['sols'][key_s]['NEB_HP_PP_angle_list']

                    nb_plotted += 1
                    rms_list.append(RMS_hp)
                    energy_list.append(energy)
                    temp_list.append(temperature_list)
                    stab_list_h.append(healthy_angle_list)
                    stab_list_p.append(prion_angle_list)
                    rc_coor_list.append(reaction_coordinate)
                    bind_coor_list.append(binded_reaction_coordinate)
                    bind_energy_list.append(binded_energy)
                    bind_pos_list.append(binded_pos_list)
                    bind_angle_list.append(binded_angle_list)
                    healthy_key.append(key_h)
                    prion_key.append(key_p)
                    sol_key.append(key_s)

In [ ]:
def plot_from_key(key_healthy, key_prion, key_sol):

    protein_pair = db.load_protein_pair(key_healthy, key_prion, key_sol)

    healthy_pos = protein_pair.healthy_position
    prion_pos = protein_pair.prion_position
    connection = protein_pair.connection

    min_x = min(min(x[1] for x in prion_pos), min(x[1] for x in healthy_pos))
    max_x = max(max(x[1] for x in prion_pos), max(x[1] for x in healthy_pos))
    min_y = min(min(x[2] for x in prion_pos), min(x[2] for x in healthy_pos))
    max_y = max(max(x[2] for x in prion_pos), max(x[2] for x in healthy_pos))


    fig, (ax1,ax2) = plt.subplots(1, 2,figsize=(8, 12), dpi=80)
    ax1.set_xlim(min_x-0.05, max_x+0.05)
    ax1.set_ylim(min_y-0.05, max_y+0.05)
    ax2.set_xlim(min_x-0.05, max_x+0.05)
    ax2.set_ylim(min_y-0.05, max_y+0.05)
    ax1.set_aspect('equal', adjustable='box')
    ax2.set_aspect('equal', adjustable='box')

    #plot
    for conn in connection:
        i, j = conn[0],conn[1]
        xl, yl = [prion_pos[i][1],prion_pos[j][1]], [prion_pos[i][2],prion_pos[j][2]]
        ax1.plot(xl, yl)
        xl, yl = [healthy_pos[i][1],healthy_pos[j][1]], [healthy_pos[i][2],healthy_pos[j][2]]
        ax2.plot(xl, yl)

    plt.show()
    
def max_none(e, min_e, max_e):
    if e is not None:
        m = max(e)
        if m>min_e and m<max_e:
            return m
    
        return None
def min_capped(min_val, val):
    return min_val if val < min_val else val

In [ ]:
################################################################
#                Plot of the activation energy                 #
################################################################



fig = plt.figure(figsize=(8, 6), dpi=120)

for i in range(len(energy_list)):
    if energy_list[i] is not None and max(energy_list[i]) < 0.1:
        energy_capped = [min_capped(1e-5, e) for e in energy_list[i]]

        plt.plot(rc_coor_list[i], energy_capped, c='b', alpha=0.01)


plt.xlabel("reaction coordinate (arbitrary unit) ")
plt.ylabel("energy (arbitrary unit)")

fig.savefig('/Users/mathieuouellet/Desktop/AMP/Figures/rc_e_all.png')
plt.show()

In [ ]:
#############################################
#         PLOT ENERGY H-P vs HP-PP          #
#############################################


min_e, max_e = 1E-8, 1E0

log_e_list = [max_none(e, min_e, max_e ) for e in energy_list]
log_binded_e = [max_none(e, min_e, max_e ) for e in bind_energy_list]

tot = sum(item is not None for item in log_binded_e)
print(tot)

fig = plt.figure(figsize=(8, 6), dpi=120)
plt.plot(log_e_list, log_binded_e, 'o', alpha=0.1)
plt.xlabel("maximum energy H<->P transition (arbitrary unit) ")
plt.ylabel("maximum energy HP<->PP transition (arbitrary unit)")
plt.ylim((1e-4, 1e-1))
plt.xlim((1e-4, 1e-1))

plt.yscale('log')
plt.xscale('log')
fig.savefig(
    '/Users/mathieuouellet/Desktop/AMP/Figures/H_stability_log_e_HP_vs_HPPP_transition.png')
plt.show()


In [ ]:
db = database.Database("/Users/mathieuouellet/Desktop/n=5_5deg_2_good")
list_potential_key = []

found = False
key_pairs = list(db.db_pairs.keys())
for k_h in key_pairs:
    keys_prion = list(db.db_pairs[k_h].keys())
    for k_p in keys_prion:
        if 'sols' in db.db_pairs[k_h][k_p] and len(db.db_pairs[k_h][k_p]['sols'].keys()) > 1:
            
            first_key = list(db.db_pairs[k_h][k_p]['sols'].keys())[0]
            list_potential_key.append((k_h, k_p, first_key))
            found += 1



In [ ]:
def avg_dist(list_pos, np_array_pos):

    return sum(math.sqrt((list_pos[i][0] - np_array_pos[i, 0]) ** 2 + (list_pos[i][1] - np_array_pos[i, 1]) ** 2) for i in range(len(list_pos)))


def get_distance(pos1, pos2):
    return math.sqrt((pos1[0]-pos2[0])**2 + (pos1[1]-pos2[1])**2)


def add_center_node(pos_h, pos_p, inside_node):
    pos_p_2 = (copy.copy(pos_p))
    pos_p_2.extend(list(zip(inside_node[2], inside_node[3])))
    all_p = np.array(pos_p_2)

    # get position with inside node for healthy
    pos_h_2 = (copy.copy(pos_h))
    pos_h_2.extend(list(zip(inside_node[0], inside_node[1])))
    all_h = np.array(pos_h_2)

    return all_h, all_p


def add_edge(dict_edge, dict_node, key1, key2, node_pos, binded, n_out):

    if key1[0] == n_out:
        key1 = (0, key1[1])
    if key2[0] == n_out:
        key2 = (0, key2[1])

    # if none of them were deleted
    if key1 in dict_node and key2 in dict_node:
        dict_edge[(key1, key2)] = get_distance(
            node_pos[key1[0]], node_pos[key2[0]])
    else:
        if key1 not in dict_node and key2 in dict_node:
            dict_edge[((binded[key1[0] % n_out], 'p'), key2)] = get_distance(
                node_pos[key1[0] % n_out], node_pos[key2[0]])
        if key1 in dict_node and key2 not in dict_node:
            dict_edge[(key1, (binded[key2[0] % n_out], 'p'))] = get_distance(
                node_pos[key1[0]], node_pos[key2[0] % n_out])


def translate_position(non_translated_pos, translation):

    translated_pos = [0]*len(non_translated_pos)
    for i in range(len(non_translated_pos)):
        if i in translation:
            translated_pos[i] = (
                non_translated_pos[translation[i], 0], non_translated_pos[translation[i], 1])
        else:
            translated_pos[i] = (non_translated_pos[i, 0],
                                 non_translated_pos[i, 1])

    return translated_pos


def transform_all_point_back(binded_from_sol, binded_to_move, solution_poly, full_point_to_transform, inverse):
    # transform the inside point by refitting the subset of bonded point for HP
    trans1_hp, data1_hp, trans2_hp, data2_hp = pu.find_good_initial_guess(
        set_pt1=binded_from_sol, set_pt2=binded_to_move, data_to_shift=full_point_to_transform)
    dist1_hp, dist2_hp = avg_dist(list(solution_poly.exterior.coords), data1_hp), avg_dist(
        list(solution_poly.exterior.coords), data2_hp)
    if not inverse:
        full_point_transformed, tr = data1_hp, trans1_hp
        if dist2_hp < dist1_hp:
            full_point_transformed, tr = data2_hp, trans2_hp
    else:
        full_point_transformed, tr = data2_hp, trans1_hp
        if dist2_hp < dist1_hp:
            full_point_transformed, tr = data1_hp, trans2_hp

    return full_point_transformed, tr


def get_dict_node(full_bottom_node, full_top_node, binded_pairs_list, n_out, translation):
    # set un the dict for the nodes
    dict_node_temp, dict_node = {}, {}
    for i in range(len(full_bottom_node)):
        dict_node_temp[(i, 'h')] = full_bottom_node[i]
    for i in range(len(full_top_node)):
        dict_node_temp[(i, 'p')] = full_top_node[i]
    # remove the loop
    del dict_node_temp[(n_out, 'h')]
    del dict_node_temp[(n_out, 'p')]
    # remvove healty binding
    for p in binded_pairs_list:
        del dict_node_temp[(p[0], 'h')]

    # translate the healthy and put in new dict
    for i in range(len(full_bottom_node)):
        if (i, 'h') in dict_node_temp:
            if i in translation:
                dict_node[(translation[i], 'h')] = dict_node_temp[(i, 'h')]
            else:
                dict_node[(i, 'h')] = dict_node_temp[(i, 'h')]
        if (i, 'p') in dict_node_temp:
            dict_node[(i, 'p')] = dict_node_temp[(i, 'p')]
    return dict_node


def get_dict_edge(full_bottom_node, dict_node, binded_dict, connection,  n_out, translation):

    # get edge list
    dict_edge = {}
    # translate the healthy of hp to fit pp
    full_healthy_hp_translated = translate_position(
        full_bottom_node, translation)
    # put the outside edge for both
    for i in range(n_out+1):
        # get the position of the two consecutive edge of the outside
        key1, key2 = (i, 'h'), ((i+1) % n_out, 'h')
        add_edge(dict_edge, dict_node, key1, key2,
                 full_healthy_hp_translated, binded_dict, n_out)
        # the second protein doesn't have any edge removed therefore we can add them all
        key1p, key2p = (i, 'p'), ((i+1) % n_out, 'p')
        add_edge(dict_edge, dict_node, key1p, key2p,
                 full_healthy_hp_translated, binded_dict,  n_out)

    # edge to the inside
    for p in connection:
        f_, t_ = p[0], p[1]
        # for the botom protein some edge are removed so we need to check
        key1, key2 = (f_, 'h'), (t_, 'h')
        add_edge(dict_edge, dict_node, key1, key2,
                 full_healthy_hp_translated, binded_dict, n_out)
        # for the top one no edge are removed
        key1p, key2p = (f_, 'p'), (t_, 'p')
        add_edge(dict_edge, dict_node, key1p, key2p,
                 full_healthy_hp_translated, binded_dict, n_out)

    return dict_edge


def count_binding(binding_list):
    count_1 = Counter([p[0] for p in binding_list])
    count_2 = Counter([p[1] for p in binding_list])
    return max([max(list(count_1.values())), max(list(count_2.values()))])


def plot(node_dict: dict, edge_dict: dict, n_out):
    for edg, dist in edge_dict.items():
        start = node_dict[edg[0]]
        end = node_dict[edg[1]]
        if edg[0][0] < n_out and edg[1][0] < n_out:
            plt.plot([start[0], end[0]], [start[1], end[1]], c="black")
        else:
            plt.plot([start[0], end[0]], [start[1], end[1]], c="red")


def plot_test(pos_h, pos_p):
    x_h, y_h, l_h = [p[0] for p in pos_h], [p[1]
                                            for p in pos_h], [str(i) for i in range(len(pos_h))]
    x_p, y_p, l_p = [p[0] for p in pos_p], [p[1]
                                            for p in pos_p], [str(i) for i in range(len(pos_p))]
    fig, ax1 = plt.subplots(figsize=(8, 6))
    plt.plot(x_h, y_h, 'o', c='g')
    plt.plot(x_p, y_p, 'o', c='r')
    for i in range(len(x_h)):
        ax1.annotate(l_h[i], (x_h[i]+0.1, y_h[i]+0.1), color='green')
        ax1.annotate(l_p[i], (x_p[i]+0.05, y_p[i]+0.05), color='red')
    plt.show()


def create_protein_pair(dict_hp, dict_pp, dict_edge_pp, n_out):

    protein_pair = protein_template.Protein_Template()
    dict_hp_key = list(dict_hp.keys())

    healthy_pos_protein_format, prion_pos_protein_format = [], []
    dict_pos_in_protein = {}

    # translate_healthy_position
    for i, key in enumerate(dict_hp_key):
        typ = 1  # inside node
        if key[0] <= n_out:
            typ = 0  # outside node
        pos_hp, pos_pp = dict_hp[key], dict_pp[key]
        dict_pos_in_protein[key] = i
        healthy_pos_protein_format.append([typ, pos_hp[0], pos_hp[1]])
        prion_pos_protein_format.append([typ, pos_pp[0], pos_pp[1]])

    conn_protein_format, conn_length_list = [], []

    # define all the edges/interaction
    for key in dict_edge_pp:
        from_id = dict_pos_in_protein[key[0]]
        to_id = dict_pos_in_protein[key[1]]
        conn_protein_format.append([from_id, to_id])
        conn_length_list.append(dict_edge_pp[key])

    # format in numpy array
    healthy_pos_protein_format = np.array(healthy_pos_protein_format)
    prion_pos_protein_format = np.array(prion_pos_protein_format)
    conn_protein_format = np.array(conn_protein_format)

    # create the protein template
    protein_pair.set_healthy_structure(
        position=healthy_pos_protein_format, connection=conn_protein_format)
    protein_pair.set_prion_structure(position=prion_pos_protein_format)
    protein_pair.set_bond_length(conn_length_list)

    return protein_pair, dict_pos_in_protein


def get_position_healthy_prion(all_position, dict_pos_in_protein, binded_pp, n_node, n_out):
    pos_bottom, pos_top = [], []

    for i in range(n_node+1):
        if i != n_out:  # remove the node that was previously used to close the outside polygon

            if (i, 'h') in dict_pos_in_protein:
                pos = all_position[dict_pos_in_protein[(i, 'h')]]
            else:
                pos = all_position[dict_pos_in_protein[(
                    binded_pp[i % n_out], 'p')]]
                print((binded_pp[i % n_out], 'p'))
            pos_bottom.append([pos[1], pos[2]])
            if (i, 'p') in dict_pos_in_protein:
                pos = all_position[dict_pos_in_protein[(i, 'p')]]
                pos_top.append([pos[1], pos[2]])

    return pos_bottom, pos_top

def euclid2(vec1, vec2):
    return(math.sqrt((vec1[0]-vec2[0])**2 +(vec1[1]-vec2[1])**2) )



def make_binded():

    k_h, k_p, k_s = list_potential_key[2]

    pos_h = db.db_pairs[k_h]['struct_healthy']
    pos_p = db.db_pairs[k_h][k_p]['struct']
    translation = db.db_pairs[k_h][k_p]['translation']
    binded_hp = db.db_pairs[k_h][k_p]['binded_hp']
    inside_node = db.db_pairs[k_h][k_p]['sols'][k_s]['inside_node']
    conn = db.db_pairs[k_h][k_p]['sols'][k_s]['conn']
    n_out = len(pos_h)-1
    # load properties
    target_previous_pp_fitness = db.db_pairs[k_h][k_p]['pp_fitness']
    target_previous_hp_fitness = db.db_pairs[k_h][k_p]['hp_fitness']
    n_point_max = db.db_parameter["n_point_max"]
    min_r_squared_default = db.min_r_squared_default

    # TODO add a check for no multi bind
    assert count_binding(binded_hp) == 1, "multiple binding"

    # correct connection (fit the indices and not matlab)
    conn = [(c[0]-1, c[1]) for c in conn]

    # get back the binding PH and the binding PP
    healthy_poly, prion_poly = Polygon(pos_h), Polygon(pos_p)
    fit_ordered_hp, sol_ordered_hp, output_index_hp, _ = pu.estimate_stacking_icp(
        healthy_poly, prion_poly, avg_dist_tol=0.3, n_point_max=n_point_max,  multiple_bind=False, output_index=True)
    hp_fit_max = pu.rescale_fit(fit_ordered_hp[-1], min_r_squared_default)
    fit_ordered_pp, sol_ordered_pp, output_index_pp, _ = pu.estimate_stacking_icp(
        prion_poly, prion_poly, avg_dist_tol=0.3, n_point_max=n_point_max,  multiple_bind=False, output_index=True)
    pp_fit_max = pu.rescale_fit(fit_ordered_pp[-1], min_r_squared_default)

    # todo assert if they are kinda the same
    assert (hp_fit_max-target_previous_hp_fitness) < 0.01, "there is a big difference in the fitness" + \
        str(hp_fit_max-target_previous_hp_fitness)
    assert (pp_fit_max-target_previous_pp_fitness) < 0.01, "there is a big difference in the fitness" + \
        str(pp_fit_max-target_previous_pp_fitness)

    # get position with inside node for prion / healthy
    all_h, all_p = add_center_node(pos_h, pos_p, inside_node)
    full_healthy_hp, full_prion_pp_p2 = np.copy(all_h), np.copy(all_p)

    # get binded node from the prion that do not move
    healthy_binded, prion_binded = [p[0]
                                    for p in binded_hp], [p[1] for p in binded_hp]
    init_prion_3 = [list(prion_poly.exterior.coords)[i] for i in prion_binded]
    # transform the inside point by refitting the subset of bonded point for HP
    sol_hp_3 = [list(sol_ordered_hp[-1].exterior.coords)[i]
                for i in prion_binded]
    full_prion_hp, trans_hp = transform_all_point_back(
        sol_hp_3, init_prion_3, sol_ordered_hp[-1], all_p, False)
    # transform the inside point by refitting the subset of bonded point for PP
    sol_pp_3 = [list(sol_ordered_pp[-1].exterior.coords)[i]
                for i in prion_binded]
    full_prion_pp_p1, trans_pp = transform_all_point_back(
        sol_pp_3, init_prion_3, sol_ordered_pp[-1], all_p, False)

    # set un the dict for the nodes
    binded_translated = {translation[p[0]]: p[1] for p in binded_hp}
    dict_hp = get_dict_node(full_healthy_hp, full_prion_hp,
                            binded_hp, n_out, translation)
    dict_edge_hp = get_dict_edge(
        full_healthy_hp, dict_hp, binded_translated, conn, n_out, translation)

    # set un the dict for the nodes
    identity_translation = {i: i for i in range(n_out)}
    binded_pp = [(translation[p[0]], p[1]) for p in binded_hp]
    dict_pp = get_dict_node(
        full_prion_pp_p2, full_prion_pp_p1, binded_pp, n_out, identity_translation)
    dict_edge_pp = get_dict_edge(
        full_prion_pp_p2, dict_pp, binded_translated, conn, n_out, identity_translation)

    assert set(dict_edge_pp.keys()) == set(
        dict_edge_hp.keys()), "edge set should be the same"
    assert set(dict_pp.keys()) == set(
        dict_hp.keys()), "node set should be the same"

    protein_pair, dict_pos_in_protein = create_protein_pair(
        dict_hp, dict_pp, dict_edge_pp, n_out)

    pos_bottom, pos_top = get_position_healthy_prion(
        protein_pair.prion_position, dict_pos_in_protein, binded_translated, len(all_h), n_out)
    print(len(protein_pair.prion_position))
    print(binded_hp)
    print(dict_pos_in_protein)
    # print(protein_pair.prion_position)
    # print(pos_bottom)
    # print(pos_top)

    print("HP")
    plot(dict_hp, dict_edge_hp, n_out)
    plt.show()
    print("PP")
    plot(dict_pp, dict_edge_pp, n_out)
    plt.show()
    print("top")
    ax1 = plt.subplot()
    plt.plot([p[0] for p in pos_top], [p[1] for p in pos_top], 'o')
    for i in range(len(pos_top)):
        p = pos_top[i]
        ax1.annotate(str(i), (p[0], p[1]), color='green')
    plt.show()
    print("down")
    ax1 = plt.subplot()
    plt.plot([p[0] for p in pos_bottom], [p[1] for p in pos_bottom], 'o')
    for i in range(len(pos_bottom)):
        p = pos_bottom[i]
        ax1.annotate(str(i), (p[0], p[1]), color='green')

    plt.show()

def get_markov(conf_mol_list_list, multiple_conf_list = False):
    count_init = [0]*3
    transtition = np.zeros((3,3))
    precision = 0.00001
    max_time_markov = 1000000

    if not multiple_conf_list:
        conf_mol_list_list =[conf_mol_list_list]
  
    for conf_mol_list in conf_mol_list_list:
        for mol_id in range(len(conf_mol_list[0])):

            conf_list = [conf[mol_id] for conf in conf_mol_list]

            for i in range(len(conf_list)-1):
                count_init[conf_list[i]]+=1
                transtition[conf_list[i],conf_list[i+1]]+=1

    for i, j in itertools.product(range(3), range(3)):
        if count_init[i]!=0:
            transtition[i,j]=transtition[i,j]/count_init[i]
        else:
            transtition[i,i] = 1


    transtition = np.transpose(transtition)
    initial_state = np.array( [[1],[0],[0]])
    active_state = initial_state

    list_prion,list_healthy,list_other = [], [], []
    list_healthy.append(active_state[0,0])
    list_prion.append(active_state[1,0])
    list_other.append(active_state[2,0])

    t_react = 0
    for t in range(max_time_markov):
        active_state = np.dot(transtition,active_state)
        list_healthy.append(active_state[0,0])
        list_prion.append(active_state[1,0])
        list_other.append(active_state[2,0])
        delta = abs(list_prion[-1]-list_prion[-2]) + abs(list_healthy[-1]-list_healthy[-2]) + abs(list_other[-1]-list_other[-2])
        if precision > delta :
            t_react = t
            break
    

    return list_healthy[-1], list_prion[-1], list_other[-1], t_react, np.transpose(transtition)

def get_markov_confidence_2x2(transtition2x2, ratio_uncertainty):


    a = transtition2x2[1,0] #rate_h_to_p
    b = transtition2x2[0,1] #rate_p_to_h
    if a==0 and b==0:
        return 0,0,0
        
    error = ratio_uncertainty* (2*a*b) /((a + b)**2) 
    
    return   a/(a+b), error
    


def get_markov_confidence(conf_mol_list, n_sim):
    count_init = [0]*3
    transtition_star = np.zeros((3,3))
    precision = 0.00001
    max_time_markov = 1000000

    for mol_id in range(len(conf_mol_list[0])):

        conf_list = [conf[mol_id] for conf in conf_mol_list]

        for i in range(len(conf_list)-1):
            count_init[conf_list[i]]+=1
            transtition_star[conf_list[i],conf_list[i+1]]+=1

    list_all_healthy, list_all_prion, list_all_other, list_all_t_react = [], [], [], []

    #for each simulation
    for _ in range(n_sim):
        
        #create a matrix 
        transtition_interval = np.zeros((3,3))
        for row in range(3):
            prop_star = [transtition_star[row,j] for j in range(3)]
            if prop_star[0]!=0 and prop_star[1]!=0 and prop_star[2]!=0:
                confint = ssp.multinomial_proportions_confint(counts=prop_star, alpha=0.05)
                new_count = [confint[i,0]+(confint[i,1]-confint[i,0])*random.random() for i in range(3)]
                new_count = [c/sum(new_count) for c in new_count]
                #print(row, transtition_star[row], new_count)
                
            elif sum(prop_star)!=0:
                new_count = [c/sum(prop_star) for c in prop_star]
            else:
                new_count = [0,0,0]
                new_count[row]=1.0
            for col in range(3):
                    transtition_interval[row,col]=new_count[col]

        transtition = np.transpose(transtition_interval)
        initial_state = np.array( [[1],[0],[0]])
        active_state = initial_state

        list_prion,list_healthy,list_other = [], [], []
        list_healthy.append(active_state[0,0])
        list_prion.append(active_state[1,0])
        list_other.append(active_state[2,0])

        t_react = 0
        for t in range(max_time_markov):
            active_state = np.dot(transtition,active_state)
            list_healthy.append(active_state[0,0])
            list_prion.append(active_state[1,0])
            list_other.append(active_state[2,0])
            delta = abs(list_prion[-1]-list_prion[-2]) + abs(list_healthy[-1]-list_healthy[-2]) + abs(list_other[-1]-list_other[-2])
            if precision > delta :
                t_react = t
                break
        
        list_all_healthy.append(list_healthy[-1])
        list_all_prion.append(list_prion[-1])
        list_all_other.append(list_other[-1])
        list_all_t_react.append(t_react)

              
    #set the star transition again
    for i, j in itertools.product(range(3), range(3)):
        if count_init[i]!=0:
            transtition_star[i,j]=transtition_star[i,j]/count_init[i]
        else:
            transtition_star[i,i] = 1

    return list_all_healthy, list_all_prion, list_all_other, list_all_t_react,  np.transpose(transtition_star)  



'\ndef make_binded():\n\n    k_h, k_p, k_s = list_potential_key[2]\n\n    pos_h = db.db_pairs[k_h][\'struct_healthy\']\n    pos_p = db.db_pairs[k_h][k_p][\'struct\']\n    translation = db.db_pairs[k_h][k_p][\'translation\']\n    binded_hp = db.db_pairs[k_h][k_p][\'binded_hp\']\n    inside_node = db.db_pairs[k_h][k_p][\'sols\'][k_s][\'inside_node\']\n    conn = db.db_pairs[k_h][k_p][\'sols\'][k_s][\'conn\']\n    n_out = len(pos_h)-1\n    # load properties\n    target_previous_pp_fitness = db.db_pairs[k_h][k_p][\'pp_fitness\']\n    target_previous_hp_fitness = db.db_pairs[k_h][k_p][\'hp_fitness\']\n    n_point_max = db.db_parameter["n_point_max"]\n    min_r_squared_default = db.min_r_squared_default\n\n    # TODO add a check for no multi bind\n    assert count_binding(binded_hp) == 1, "multiple binding"\n\n    # correct connection (fit the indices and not matlab)\n    conn = [(c[0]-1, c[1]) for c in conn]\n\n    # get back the binding PH and the binding PP\n    healthy_poly, prion_poly =

# 5. **Dynamical Simulation**

## Simulation and Protein Binding Documentation

### Overview
This script is designed to handle simulations involving protein binding and molecular interactions using LAMMPS. It generates, transforms, and evaluates protein structures within a defined computational framework. The script includes functionalities for molecule insertion, energy minimization, and dynamic simulation using LAMMPS and mathematical utilities.

### Key Functions

#### 1. `get_protein_pairs_binded_divided_bypass()`
- Transforms and aligns protein structures while maintaining binding constraints.
- Returns transformed protein configurations with updated molecular positions.

#### 2. `get_protein_pairs_binded_divided()`
- Computes protein structure alignments and fitness scores.
- Ensures that healthy and prion protein structures are correctly matched.
- Returns transformed protein positions with connectivity data.

#### 3. `set_up_molecule_simulation()`
- Initializes a LAMMPS simulation with specified molecular configurations.
- Defines interaction parameters such as bond types, Lennard-Jones potentials, and simulation boundaries.
- Ensures compatibility with molecular binding constraints.

#### 4. `insert_avoiding_molecule()`
- Places molecules into a simulation box while avoiding overlap.
- Uses quasi-random sampling for uniform distribution.
- Assigns molecules to appropriate groups for simulation tracking.

#### 5. `insert_avoiding_binded_molecule()`
- Similar to `insert_avoiding_molecule()` but ensures molecular pairs remain bound during placement.
- Introduces relative translations between molecule pairs.

#### 6. `simulate_HP()`
- Runs a LAMMPS simulation on healthy and prion binded structures.
- Logs molecular interactions and connectivity over time.
- Generates output files and optionally produces visualization GIFs.

#### 7. `simulate()`
- Iterates over a temperature range to perform multiple simulations.
- Stores results for healthy and binded protein states.
- Saves all computed values for further analysis.

### Usage
The script is typically executed with user-defined parameters. Example:

```python
if __name__ == "__main__":
    path = "path/to/database"
    db = database.Database(path)
    key_ps_pair = None  # Simulate all solutions
    list_temp = None  # Use standard temperature range
    sim_param = None  # Default simulation parameters
    
    for key_h, key_p, key_s in key_ps_pair:
        worker_id = os.getpid()
        simulate(db, worker_id, (key_h, key_p, key_s),
                 num=NB_TEMP,
                 temperature_min=MIN_TEMP,
                 temperature_max=MAX_TEMP,
                 PRE=PRE,
                 is_PP=False,
                 temperature_list=list_temp,
                 sim_param=sim_param)
```




# Code for the Markov analysis

In [ ]:

def get_prion_property(temp_list, p_list, temp_list_H, p_list_H, o_list_H):

    min_p_healthy = 0.05 #healthy should not become prion
    min_o_binded = 0.05 #side product should not be produced 

    list1 = [t for i,t in enumerate(temp_list_H) if p_list_H[i]< min_p_healthy]
    list2 = [t for i,t in enumerate(temp_list_H) if o_list_H[i]< min_o_binded]

    max_p, id_p = -1, -1

    if len(list1)*len(list2)>0:
        sep_p_0 = max(list1)
        bound_o_0 = max(list2)
        bound_t = min(sep_p_0,bound_o_0)

        for i,t in enumerate(temp_list):
            if p_list[i]>=max_p and t<=bound_t:
                id_p=i
                max_p=p_list[i]

    return (temp_list[id_p],max_p) if id_p!=-1 else (None, None)



def get_conformation_from_saved(path, saved=None, binded_dict=None, nonbinded_dict=None):
    
    if saved is None:
        saved = Save_file_full_sim.load(path)

    if binded_dict is None :
        binded_dict = {}
        nonbinded_dict = {}

    print(saved.key)
    key_saved = saved.key
    
    #get time where we start to loose non-binded-healthy 
    for i in range(len(saved.temp_list_H)):
        #print(len(saved.count_list_H[i]))
        if len(saved.count_list_H[i])==8:
            count_healthy_H, count_prion_H, count_other_H, count_healthy_even_H, count_prion_even_H, count_other_even_H, conf_mol_list_H, distance_list_H  = saved.count_list_H[i]
        else:
            count_healthy_H, count_prion_H, count_other_H, count_healthy_even_H, count_prion_even_H, count_other_even_H, conf_mol_list_H  = saved.count_list_H[i]
            distance_list_H=None
        
        temp = saved.temp_list_H[i]
        if temp not in nonbinded_dict:
            nonbinded_dict[temp] = []
        nonbinded_dict[temp].append(conf_mol_list_H)


    #get time where we start to loose binded-healthy 
    for i in range(len(saved.temp_list)):

        if len(saved.count_list[i])==8:
            count_healthy, count_prion, count_other, count_healthy_even, count_prion_even, count_other_even, conf_mol_list, distance_list = saved.count_list[i]
        else:
            count_healthy, count_prion, count_other, count_healthy_even, count_prion_even, count_other_even, conf_mol_list = saved.count_list[i]
            distance_list = None

        temp = saved.temp_list[i]
        if temp not in binded_dict:
            binded_dict[temp] = []
        binded_dict[temp].append(conf_mol_list)
    
    return nonbinded_dict, binded_dict

def merge_dict(dict1, dict2):
    for key in dict2.keys():
        if key not in dict1:
            dict1[key] = []
        dict1[key].extend(dict2[key])

def get_markov_binned(conformation_dict, temperature_range):

    conformation_dict_binned = {i:[] for i in range(len(temperature_range))}
    t_list, p_list, o_list, h_list  = [],[],[],[]
    rate_diff = []



    for temp in conformation_dict:
        for conf_list in conformation_dict[temp]:

            dist, best_t_ind = 10000000000,  0
            for t_ind in range(len(temperature_range)):
                if abs(temp-temperature_range[t_ind]) < dist:
                    dist = abs(temp-temperature_range[t_ind])
                    best_t_ind = t_ind
                
            
            conformation_dict_binned[best_t_ind].append(conf_list)

    for t_ind in range(len(temperature_range)):

        if len(conformation_dict_binned[t_ind]) > 0:
            h,p,o,t_react, t_mat = get_markov(conformation_dict_binned[t_ind], multiple_conf_list=True)
            val,error = get_markov_confidence_2x2(np.transpose(t_mat), 0.5)

            t_list.append(temperature_range[t_ind])
            h_list.append(h)
            p_list.append(p)
            o_list.append(o)
            rate_diff.append([val,error ])#t_mat[0,1] - t_mat[1,0])

    return t_list, h_list, p_list, o_list, rate_diff
    

def plot_binded(db, key_selected, ax):
    
    k_h, k_p, k_s = key_selected
    pos_h = db.db_pairs[k_h]['struct_healthy']
    pos_p = db.db_pairs[k_h][k_p]['struct']
    translation = db.db_pairs[k_h][k_p]['translation']
    binded_hp = db.db_pairs[k_h][k_p]['binded_hp']
    inside_node = db.db_pairs[k_h][k_p]['sols'][k_s]['inside_node']
    conn = db.db_pairs[k_h][k_p]['sols'][k_s]['conn']
    n_out = len(pos_h)-1
    # load properties
    target_previous_pp_fitness = db.db_pairs[k_h][k_p]['pp_fitness']
    target_previous_hp_fitness = db.db_pairs[k_h][k_p]['hp_fitness']
    n_point_max = db.db_parameter["n_point_max"]
    min_r_squared_default = db.min_r_squared_default

    

    if not 'hp_fit_dic' in  db.db_pairs[k_h][k_p].keys() :

        full_healthy_hp, full_prion_hp, prion_binded, full_prion_pp_p1 = simulation.get_protein_pairs_binded_divided(pos_h, pos_p, binded_hp, inside_node, translation, target_previous_pp_fitness, target_previous_hp_fitness, min_r_squared_default, n_point_max, n_out)

    else:
        hp_fit_dic = db.db_pairs[k_h][k_p]['hp_fit_dic']
        pp_fit_dic = db.db_pairs[k_h][k_p]['pp_fit_dic']
        hp_vec = hp_fit_dic['sol_ordered_hp'][-1], hp_fit_dic['binded_hp']
        pp_vec = pp_fit_dic['sol_ordered_pp'][-1], pp_fit_dic['binded_pp']
        full_healthy_hp, full_prion_hp, prion_binded, full_prion_pp_p1 = simulation.get_protein_pairs_binded_divided_bypass(pos_h, pos_p, translation, inside_node, conn=[], hp_vec=hp_vec, pp_vec=pp_vec, n_out=len(pos_h)-1, translated=False )


    all_conn = [(p[0]-1,p[1]-1) for p in conn] + [(i,i+1) for i in range(n_out-1)] + [(n_out-1,0)]
    all_conn = np.array(all_conn)
    
    protein_pair = protein_template.Protein_Template()
    protein_pair.set_prion_structure(full_prion_hp, connection = all_conn)
    protein_pair.set_healthy_structure(full_healthy_hp)
    protein_pair.plot(ax)

    return protein_pair

def get_neb_pos_list(vec_out, zero_on = 0, rotate_on = 1):
    
    pos_h_x = [[p-v[0][zero_on] for p in v[0]] for v in vec_out]
    pos_h_y = [[p-v[1][zero_on] for p in v[1]] for v in vec_out]
    pos_list = []

    for frame in range(len(pos_h_x)):
        theta = math.atan2(pos_h_y[frame][rotate_on], pos_h_x[frame][rotate_on])
        c, s = np.cos(-theta), np.sin(-theta)
        R = np.array(((c, -s), (s, c)))
        pos_array = [ np.dot(R,np.array([x,y])) for x,y in zip(pos_h_x[frame],pos_h_y[frame]) ] 
        pos_array = [ np.dot(np.array([[1.0/pos_array[rotate_on][0],0],[0,1.0/pos_array[rotate_on][0]]]),arr) for arr in pos_array]
        pos_list.append([v[0] for v in pos_array] + [v[1] for v in pos_array])

    return np.array(pos_list)

def _translate(all_pos, dict_pos_in_protein, binded_translated, n_unbind, n_out):

    list_pos_top = []
    list_pos_bottom = []

    for j in range(len(all_pos)):



        pos_bottom, pos_top = NEB.get_position_healthy_prion(
            [[all_pos[j][0][i], all_pos[j][1][i]] for i in range(len(all_pos[j][0]))], dict_pos_in_protein, binded_translated, n_unbind, n_out)
        
        pos_top = list(zip(*pos_top))
        pos_bottom = list(zip(*pos_bottom))

        list_pos_top.append(pos_top)
        list_pos_bottom.append(pos_bottom)

    return  list_pos_bottom, list_pos_top

def get_translation_binded_function_neb(db, key_healthy, key_prion, key_sol):

    value_dict = NEB_binded_main.load_value(db, None, key_healthy, key_prion, key_sol)

    n_out = len(value_dict["pos_h"])-1
    # load the protein
    protein_pair_unbinded = db.load_protein_pair(
        key_healthy, key_prion, key_sol)
        
    if not 'hp_fit_dic' in  db.db_pairs[key_healthy][key_prion].keys():
        protein_pair_binded, dict_pos_in_protein, (in_edge,out_edge)  = NEB.get_protein_pairs_binded(value_dict["pos_h"], value_dict["pos_p"], value_dict["translation"], binded_hp=value_dict["binded_hp"], inside_node=value_dict["inside_node"], conn=value_dict["conn"], target_previous_pp_fitness=value_dict["target_previous_pp_fitness"],
                                                                            target_previous_hp_fitness=value_dict["target_previous_hp_fitness"], min_r_squared_default=value_dict["min_r_squared_default"], n_point_max=value_dict["n_point_max"])
    else:
        print("bypass in angle")
        hp_fit_dic = db.db_pairs[key_healthy][key_prion]['hp_fit_dic']
        pp_fit_dic = db.db_pairs[key_healthy][key_prion]['pp_fit_dic']
        hp_vec = hp_fit_dic['sol_ordered_hp'][-1], hp_fit_dic['binded_hp']
        pp_vec = pp_fit_dic['sol_ordered_pp'][-1], pp_fit_dic['binded_pp']
        protein_pair_binded, dict_pos_in_protein, edges = NEB.get_protein_pairs_binded_bypass(value_dict["pos_h"], value_dict["pos_p"], value_dict["translation"], inside_node=value_dict["inside_node"], conn=value_dict["conn"], hp_vec=hp_vec, pp_vec=pp_vec)
    
    
    # get translation
    binded_translated = {
        value_dict["translation"][p[0]]: p[1] for p in value_dict["binded_hp"]}

    pos_unbind = [[], []]
    for i in range(len(protein_pair_unbinded.healthy_position)):
        pos = protein_pair_unbinded.healthy_position[i]
        pos_unbind[0].append([pos[1], pos[2]])
        pos = protein_pair_unbinded.prion_position[i]
        pos_unbind[1].append([pos[1], pos[2]])
    
    trans_func = partial(_translate, dict_pos_in_protein=dict_pos_in_protein, binded_translated=binded_translated, n_unbind=len(protein_pair_unbinded.prion_position), n_out=n_out)

    return  trans_func



# Plot transition between structures

In [ ]:

def plot_gif(energy_list : list[float], conn_all, pos_all, energy_all, box, gif_name : str,  list_of_plot : list[dict] = [],  speed_up_zero = 4):
    # sourcery no-metrics skip: default-mutable-arg

    conn_in, conn_out = conn_all
    in_pos, out_pos = pos_all
    in_energy_length, out_energy_length = energy_all


    cmap  = plt.get_cmap('cividis')
    max_energy = max(energy_list)
    
    filenames = []
    selector = [0]*len(energy_list)
    for i in range(len(energy_list)):
        if energy_list[i] < 0.01 * max(energy_list) and i > 0:
            selector[i] = selector[i-1]+1


    for i in range(len(energy_list)):
        
        #if select 
        if selector[i]==0 or selector[i]%speed_up_zero==0 or i==len(energy_list)-1:
            height_fig = 3 + len(list_of_plot) + 1 
            height_ratio = [1]*(len(list_of_plot)+1) + [3]

            fig, ax_list = plt.subplots(len(list_of_plot)+2, 1,figsize=(8, height_fig*3), dpi=300, gridspec_kw={'height_ratios': height_ratio})

            ax_list[-1].set_xlim(box[0]-0.05, box[1]+0.05)
            ax_list[-1].set_ylim(box[2]-0.05, box[3]+0.05)
            ax_list[-1].set_aspect('equal', adjustable='box')

            for j in range(len(conn_out)):
                xl, yl = out_pos[i][j]
                ener = out_energy_length[i][j]
                ax_list[-1].plot(xl, yl, c=cmap(ener/max_energy))

            for j in range(len(conn_in)):
                break
                xl, yl = in_pos[i][j]
                ener = in_energy_length[i][j]
                ax_list[-1].plot(xl, yl, c=cmap(ener/max_energy))

            #plot energy
            xl = [k/len(energy_list) for k in range(i+1) ]
            yl = energy_list[:i+1]
            ax_list[0].plot(xl,yl, linewidth=3, label = 'average bead')
            ax_list[0].set_xlim((0,1))
            ax_list[0].set_ylim((0,1.2*max_energy))
            ax_list[0].set_xlabel("reaction coordinate")
            ax_list[0].set_ylabel("energy")



            #plot the rest
            for i_p, to_plot in enumerate(list_of_plot):
                min_y, max_y = 1E100, -1E100
                
                for yl_full, lab in zip(to_plot["ys"], to_plot["labels"]) : 

                    xl = [k/len(energy_list) for k in range(i+1) ]
                    yl = yl_full[:i+1]
                    ax_list[i_p+1].plot(xl,yl, linewidth=3, label = lab)
                    if min(yl_full)<min_y:
                        min_y= min(yl_full)
                    if max(yl_full)>max_y:
                        max_y = max(yl_full)
                    
                ax_list[i_p+1].set_xlim((0,1))
                ax_list[i_p+1].set_ylim((1.2*min_y,1.2*max_y))
                ax_list[i_p+1].set_ylabel(to_plot["ylabel"])


            #make multiple frame for the beginning and the end of the animation
            max_k=1
            if i in [0, len(energy_list) - 1]:
                max_k=10
            #run over the duplicate frames
            for k in range(max_k):
                filename = f'{i}_{k}.png'
                filenames.append(filename)

                # save frame
                plt.savefig(filename)
            plt.close()
            plt.show()

    # build gif
    gif_path = f'gif_out/{max(energy_list)}_{gif_name}.gif'
    with imageio.get_writer(gif_path, mode='I') as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    # Remove files
    for filename in set(filenames):
        os.remove(filename)

    return gif_path

def interpret_NEB_atom_list_H_P(energy_list, atom_list_list, conn, n_out):
    # sourcery no-metrics skip: default-mutable-arg
    
    print(conn)
    in_connection_length, out_connection_length  = [], [] #expected length for each connection
    min_x, max_x, min_y, max_y = 1E10,-1E10,1E10,-1E10
    out_energy_length, in_energy_length = [], [] #energy for each connection
    out_pos, in_pos = [], []

    for i in range(len(energy_list)):
        
        #check if we just receive inside connection from conn or conn_in and out 
        if len(conn)==2:
            conn_in, conn_out = conn
        else:  
            conn_in, conn_out = conn, [ (i,(i+1)%n_out ) for i in range(n_out)]

        out_connection_list, out_active_connection_length =[], []
        for  c in conn_out:
            x1,x2 = atom_list_list[i][0][c[0]], atom_list_list[i][0][c[1]]
            y1,y2 = atom_list_list[i][1][c[0]], atom_list_list[i][1][c[1]]
            xl,yl = [x1, x2] , [y1, y2]
            min_x = min(min_x, x1, x2)
            min_y = min(min_y, y1, y2)
            max_x = max(max_x, x1, x2)
            max_y = max(max_y, y1, y2)
            out_connection_list.append( [xl, yl] )
            if i == 0:
                out_connection_length.append(math.sqrt((x1-x2)**2 + (y1-y2)**2))
            out_active_connection_length.append(math.sqrt((x1-x2)**2 + (y1-y2)**2))

        energy_edge_out = [(out_connection_length[k] - out_active_connection_length[k])**2 for k in range(len(conn_out))]
        out_energy_length.append(energy_edge_out)
        out_pos.append(out_connection_list)

        in_active_connection_length, in_connection_list = [], []
        for  c in conn_in:
            x1,x2 = atom_list_list[i][0][c[0]], atom_list_list[i][0][c[1]]
            y1,y2 = atom_list_list[i][1][c[0]], atom_list_list[i][1][c[1]]
            xl,yl = [x1, x2] , [y1, y2]
            min_x = min(min_x, x1, x2)
            min_y = min(min_y, y1, y2)
            max_x = max(max_x, x1, x2)
            max_y = max(max_y, y1, y2)
            in_connection_list.append( [xl,yl])
            if i == 0:
                in_connection_length.append(math.sqrt((x1-x2)**2 + (y1-y2)**2))
            in_active_connection_length.append(math.sqrt((x1-x2)**2 + (y1-y2)**2))

        energy_edge_in = [(in_connection_length[k] - in_active_connection_length[k])**2 for k in range(len(conn_in))]
        in_energy_length.append(energy_edge_in)
        in_pos.append(in_connection_list)

    return  (in_energy_length, out_energy_length), (in_pos, out_pos), (conn_in,conn_out), (min_x, max_x, min_y, max_y)


# paths list is given by [PRE, PATH_DUMP, PATH_SCR, PATH_NEB, PATH_FINAL]
def create_gif_transition_H_P(n_out, db, key_list, paths, full_output = False):

    path_list = []
    for keys in key_list:

        NEB_main.NB_NODE = 20
        NEB_main.TIMESTEP = 0.001
        NEB_main.SPRING_INTER_REPLICA = 10000

        reaction_coordinate_list, energy_list, atom_list_list = NEB_main.get_NEB(
            db, str(0),keys[0],keys[1], keys[2], 
            PRE=paths[0], PATH_DUMP=paths[1], PATH_SCR=paths[2], PATH_NEB=paths[3], PATH_FINAL=paths[4])
        

        conn = db.db_pairs[keys[0]][keys[1]]['sols'][keys[2]]['conn']
        conn = [[x[0]-1, x[1]-1] for x in conn]
        protein_pair = db.load_protein_pair(keys[0],keys[1], keys[2])
        protein_pair.plot( axes = None, same_plot = True)

        energy_tuple, pos_tuple, conn_tuple, box = interpret_NEB_atom_list_H_P(energy_list, atom_list_list, conn, n_out)
        
        #to_plot_test = {"ys":[energy_list, list(range(len(energy_list)))], "labels":["a.","b"], "ylabel":"test" }
        gif_path = plot_gif(energy_list, conn_tuple, pos_tuple, energy_tuple, box, gif_name=f"n_{str(keys)}", list_of_plot=[], speed_up_zero = 1)
        path_list.append(gif_path)

    if full_output:
        return path_list, (reaction_coordinate_list, energy_list, atom_list_list)

    return path_list


# paths list is given by [PRE, PATH_DUMP, PATH_SCR, PATH_NEB, PATH_FINAL]
def create_gif_transition_HP_PP(n_out, db, key_list, paths, full_output = False):

#
#   IF IT DOESNT RUN, RUN THE SCRIPT IN TERMINAL TO GET THE CREDETIAL 
#
    path_list = []
    for keys in key_list:

        NEB_binded_main.NB_NODE = 14
        NEB_binded_main.TIMESTEP =  0.01
        NEB_binded_main.SPRING_INTER_REPLICA = 10000

        reaction_coordinate_list, energy_list, atom_list_list, protein_pair, edges = NEB_binded_main.get_NEB(
            db, str(0),keys[0],keys[1], keys[2], 
            PRE=paths[0], PATH_DUMP=paths[1], PATH_SCR=paths[2], PATH_NEB=paths[3], PATH_FINAL=paths[4])

 

        conn_in, conn_out = edges 
        
        energy_tuple, pos_tuple, conn_tuple, box = interpret_NEB_atom_list_H_P(energy_list, atom_list_list, (conn_in, conn_out), n_out)

        gif_path = plot_gif(energy_list, conn_tuple, pos_tuple, energy_tuple, box, gif_name=f"binded_{str(keys)}", speed_up_zero = 1)
        path_list.append(gif_path)
        
    if full_output:
        return path_list, (reaction_coordinate_list, energy_list, atom_list_list)

    return path_list


In [ ]:
# Code for plotting transition between two structure 


PRE = ''
PATH_DUMP = 'temp/'
PATH_SCR = 'lammps_util/script/'
PATH_NEB = 'lammps_util/script/neb_task.lj'
PATH_FINAL = 'lammps_util/script/prion_neb_pos.lj'

keys = (1737806584325749351, -8771402139612191995, -4281357692739835561)


output_path_list, (reaction_coordinate_list_binded, energy_list_binded, atom_list_list_binded) = create_gif_transition_HP_PP(5, db, [keys], [PRE, PATH_DUMP, PATH_SCR, PATH_NEB, PATH_FINAL], full_output = True)
#output_path_list, vec_out = create_gif_transition_H_P(5, db, [keys], [PRE, PATH_DUMP, PATH_SCR, PATH_NEB, PATH_FINAL], full_output = True)


trans_func_test = get_translation_binded_function_neb(db, keys[0], keys[1], keys[2])
list_pos_bottom, list_pos_top = trans_func_test(atom_list_list_binded)

for j in range(len(list_pos_bottom)):
    plt.plot(*list_pos_bottom[j])

# Plot transition rate, population simulation + structure (FIG.3)

In [ ]:
def save_for_R(sol_dick,key, binded, temp_list, path, is_plot = False ):

    if not isinstance(temp_list, list):
        temp_list = [temp_list]

    data = {}
    
    max_lenght = 0
    for temp in temp_list:
        n_sample = len(sol_dick[key][binded][temp])
        for s in range(n_sample):
            n_ik = len(sol_dick[key][binded][temp][s][0])
            for ik in range(n_ik):
                if ik%2==0 or binded==0:
                    l = len([sol_dick[key][binded][temp][s][k][ik] for k in range(len(sol_dick[key][binded][temp][s]))])
                    if l  > max_lenght:
                        max_lenght = l 

    run_nb = 0
    for temp in temp_list:
        n_sample = len(sol_dick[key][binded][temp])
        for s in range(n_sample):
            n_ik = len(sol_dick[key][binded][temp][s][0])
            for ik in range(n_ik):
                if ik%2==0 or binded==0:
                    data_temp = [sol_dick[key][binded][temp][s][k][ik] for k in range(len(sol_dick[key][binded][temp][s]))]
                    if len(data_temp)==max_lenght:
                        data[f'run_nb{run_nb}'] = data_temp
                        run_nb+=1
    
    
    df = pd.DataFrame(data)
    if is_plot:
        df.plot(subplots=False, figsize=(16,16))
        plt.plot()
    
    with open(path, 'wb') as f:
        feather.write_feather(df, f)

    return max_lenght

def compute_markov(sol_dick, key, binded ,path, merge_close=0.95):

    x_t = []
    p_t = []
    r_12 = []
    dr_12 = []
    r_21 = []
    dr_21 = []
    dp_t = []

    ratio = [merge_close,1/merge_close] 
    distinct_temp_dict = {}
    
    for key_temp in sol_dick[key][binded].keys():
        closest_temp = -100000
        for distinct_key in distinct_temp_dict.keys():
            if float(distinct_key)/float(key_temp) > ratio[0] and float(distinct_key)/float(key_temp) < ratio[1]:
                closest_temp = distinct_key
        
        if closest_temp!=  -100000:
            distinct_temp_dict[closest_temp].append(key_temp)
        else:
            distinct_temp_dict[key_temp] = [key_temp]



    for distinct_temp in distinct_temp_dict.keys():
        #sol_dick[key][binded].keys():


    
        max_length = save_for_R(sol_dick,key, binded, distinct_temp_dict[distinct_temp], path, is_plot=False )
        process = subprocess.Popen (f"Rscript --vanilla markov.R  /Users/mathieuouellet/Desktop/AMP/AMP/src/polygon/test.feather", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        out, err = process.communicate()

        out_val = [ float(x) for x in str(out).split('\"')[1].split(",")]
        a,b = out_val[1], out_val[2]
        da,db = out_val[5], out_val[6]
        r_12.append(a)
        dr_12.append(da)
        r_21.append(b)
        dr_21.append(db)
        

        #get error if no transition prion->healthy found bu healthy->prion found 
        if r_21[-1]==0 and r_12[-1]!=0:
            nb_step_prion = 0

            for temp in distinct_temp_dict[distinct_temp]:
                n_sample = len(sol_dick[key][binded][temp])
                for s in range(n_sample):
                    n_ik = len(sol_dick[key][binded][temp][s][0])
                    for ik in range(n_ik):
                        run = [sol_dick[key][binded][temp][s][k][ik] for k in range(len(sol_dick[key][binded][temp][s]))]
                        if len(run) == max_length:
                            nb_step_prion+= len([v for v in run if v==1])

            dr_21[-1] =  1/nb_step_prion if nb_step_prion>0 else 1
            db = dr_21[-1]
            
        dp =  math.sqrt((b**2 * da**2 + a**2 * db**2)/((a + b+1E-10)**4) )
        x_t.append(distinct_temp)
        p_t.append(a/(a+b+1E-10))
        dp_t.append(dp)


    return (x_t, p_t, dp_t), (r_12,r_21, dr_12, dr_21)

    



def load_pickle_full_sim_from_dump(path_folder, dump_done, dump_dick, database, nb_int, nb_out):

    onlyfiles = [f for f in listdir(path_folder) if isfile(join(path_folder, f))]

    print(len(onlyfiles))
    NJob = 12

    file_todo_list = [file_name for file_name in onlyfiles if file_name not in dump_done and file_name!=".DS_Store"]
    job_list = []


    print("number of job todo: " + str(len(file_todo_list)))

    current_job = []
    for file_name in file_todo_list:
        splitted = file_name.split('_')
        keys = ( int(splitted[2]), int(splitted[3]),int(splitted[4]) )
        isbinded = eval(splitted[5])
        temp = float(splitted[6]) 
        key_h, key_p, key_s = keys

        if (key_h, key_p, key_s) not in dump_dick:
            (func_translate, func_translate_binded), (endpoint_pca, endpoint_pca_bind) = get_reaction_coordinate_function(database, keys, nb_int, nb_out)
            dump_dick[(key_h, key_p, key_s)] = [{},{},((func_translate, func_translate_binded), (endpoint_pca, endpoint_pca_bind))]
            print("translation func added")

        data_for_task = (path_folder+file_name, dump_dick[(key_h, key_p, key_s)][2], (temp, isbinded, file_name, keys))
        if len(current_job)<NJob:
            current_job.append(data_for_task)
        else:
            job_list.append(current_job)
            current_job = []
            
    if len(current_job)>0:
        job_list.append(current_job)

    pool = Pool(processes=6)


    for job in job_list:
        dill.settings['recurse'] = True
        
        # call the function for each item in parallel
        payload = [ dill.dumps(j) for j in job ] 
        
        try:
            result = pool.map(dumpWorker.run_dill_encoded, payload)

            for i in range(len(result)):
                temp = job[i][2][0]
                isbinded = job[i][2][1]
                conf_list = np.transpose(np.array(result[i]))
                filename = job[i][2][2]
                keys = job[i][2][3]
                dump_done.append(filename)
                print(filename)


                if isbinded:
                    if temp not in dump_dick[keys][1]:
                        dump_dick[keys][1][temp] = [conf_list]
                    else:
                        dump_dick[keys][1][temp].append(conf_list)
                else:
                    if temp not in  dump_dick[keys][0]:
                        dump_dick[keys][0][temp] = [conf_list]
                    else:
                        dump_dick[keys][0][temp].append(conf_list)
        except Exception as e:
            print(e)
            pool.close()
        #list(my_map(pool, dumpWorker.get_conformation_from_DUMP, job))
        



    


def load_pickle_full_sim(path_folder, file_done, sol_dick):

    onlyfiles = [f for f in listdir(path_folder) if isfile(join(path_folder, f))]

    print(len(onlyfiles))
    for file_name in onlyfiles:
        if file_name not in file_done:

            spl_key = re.split('\(|\)',file_name)[1]
            key_h, key_p, key_s = [int(v) for v in re.split(',',spl_key)]
            print(key_h, key_p, key_s, sep=',')
            
            if (key_h, key_p, key_s) not in sol_dick:
                sol_dick[(key_h, key_p, key_s)] = [{},{}]

            file_done.append(file_name)
            print(path_folder, file_name)
            saved = Save_file_full_sim.load(path_folder + file_name)


            nonbinded_dict, binded_dict =  get_conformation_from_saved("None", saved=saved, binded_dict=None, nonbinded_dict=None)
            
            data_non_binded_dict = sol_dick[(key_h, key_p, key_s)][0]
            data_binded_dict = sol_dick[(key_h, key_p, key_s)][1]
            
            merge_dict(data_non_binded_dict, nonbinded_dict)
            merge_dict(data_binded_dict, binded_dict)
            sol_dick[(key_h, key_p, key_s)] = (data_non_binded_dict, data_binded_dict)


def plot_corr_prion(datab, sol_dick, plot_list=[], threshold = 0.4, is_plot = False, func_from_key = None):

    list_temp = []
    list_bind_delta = []
    list_non_bind_delta = []
    ratio_bind_energy = []
    ratio_bind_temperature = []
    plotted_nb = 0
    non_plotted_nb = 0

    list_key_plot = []
    list_full_data = []

    for key in plot_list :

        plotted_nb+=1
        list_key_plot.append(key)

        path = "/Users/mathieuouellet/Desktop/AMP/AMP/src/polygon/test.feather"
        (x_t, p_t, dp_t), (r_12,r_21,dr_12,dr_21) = compute_markov(sol_dick, key, 0, path)
        (x_t_b, p_t_b, dp_t_b), (r_12_b,r_21_b,dr_12_b,dr_21_b) = compute_markov(sol_dick, key, 1, path)

        x_t, p_t, dp_t, r_12, r_21, dr_12, dr_21 = (list(t) for t in zip(*sorted(zip(x_t, p_t, dp_t, r_12, r_21, dr_12, dr_21))))
        x_t_b, p_t_b, dp_t_b, r_12_b, r_21_b, dr_12_b, dr_21_b = (list(t) for t in zip(*sorted(zip(x_t_b, p_t_b, dp_t_b, r_12_b, r_21_b, dr_12_b, dr_21_b))))


        list_full_data.append([(x_t, p_t, dp_t), (r_12,r_21,dr_12,dr_21),(x_t_b, p_t_b, dp_t_b), (r_12_b,r_21_b,dr_12_b,dr_21_b)])

        index_binded = next((i for i, x in enumerate(p_t_b) if x>threshold ), -1)
        index_non_binded = next((i for i, x in enumerate(p_t) if x>threshold ), -1)

        firt_t_binded = x_t_b[index_binded]
        firt_t_non_binded = x_t[index_non_binded]

        delta_binded = (x_t_b[index_binded+1] -  x_t_b[index_binded-1])/4
        delta_non_binded = (x_t[index_non_binded+1] -  x_t[index_non_binded-1])/4

        #print(*key,firt_t_non_binded,firt_t_binded, sep=',')
        list_temp.append((firt_t_non_binded,firt_t_binded,delta_non_binded,delta_binded ))

        binded_energy = datab.db_pairs[key[0]][key[1]]['sols'][key[2]]['NEB_HP_PP_energy']
        free_energy = datab.db_pairs[key[0]][key[1]]['sols'][key[2]]['NEB_HP_energy']
        delta_binded = max(binded_energy)-binded_energy[0]
        delta_free = max(free_energy)-free_energy[0]
        list_bind_delta.append(delta_binded)
        list_non_bind_delta.append(delta_free)
        ratio_bind_energy.append(delta_binded/delta_free)
        ratio_bind_temperature.append(firt_t_binded/firt_t_non_binded)
        print(key)
        print(datab.db_pairs[key[0]][key[1]]['sols'][key[2]]['conn'])
        print(func_from_key(key), list_temp[-1][1], list_temp[-1][0], list_temp[-1][1]-list_temp[-1][0] )

        if is_plot:

            fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
            #(x_t, p_t, dp_t)x_t_b, p_t_b, dp_t_b)
            ax[0,0].set_xscale('log')
            ax[0,0].errorbar(x_t_b, [1-p for p in p_t_b],   yerr=dp_t_b, fmt='X', c='g', label = "healthy (bind)")
            ax[0,0].errorbar(x_t_b, p_t_b,                   yerr=dp_t_b, fmt='X', c='r', label = "prion (bind)")
            #ax[0,0].plot(t_list, o_list_bind, 'o', c='y', label="other (bind)")
            ax[0,0].legend()

            ax[0,1].errorbar(x_t, [1-p for p in p_t], yerr=dp_t, fmt='x', c='g', label = "healthy (free)")
            ax[0,1].errorbar(x_t, p_t,                yerr=dp_t, fmt='x', c='r', label = "prion (free)")
            #ax[0,1].plot(t_list_bind, o_list, 'x', c='y', label="other (free)")
            ax[0,1].legend()
            ax[0,1].set_xscale('log')

            ax[1,0].set_xscale("log")
            ax[1,0].set_yscale('log')
            #cap for the plot for the infinite uncertainty one
            dr_12 = [ (dval if dval<1000*val else 0) for val,dval in zip(r_12, dr_12) ]
            dr_21 = [ (dval if dval<1000*val else 0) for val,dval in zip(r_21, dr_21) ]
            dr_12_b = [ (dval if dval<1000*val else 0) for val,dval in zip(r_12_b, dr_12_b) ]
            dr_21_b = [ (dval if dval<1000*val else 0) for val,dval in zip(r_21_b, dr_21_b) ]

            ax[1,0].errorbar(x_t, r_12,  yerr=dr_12,        fmt='x', c='r', label="r_12")
            ax[1,0].errorbar(x_t, r_21,  yerr=dr_21,        fmt='x', c='g', label="r_21")
            ax[1,0].errorbar(x_t_b, r_12_b,  yerr=dr_12_b,    fmt='X', c='r', label="r_12_b")
            ax[1,0].errorbar(x_t_b, r_21_b,  yerr=dr_21_b,    fmt='X', c='g', label="r_21_b")
            ax[1,0].legend()

            plot_binded(datab, key, ax[1,1])
            plt.savefig(f'./figure/test/rates_{key[0]}_{key[1]}_{key[2]}_papuche.pdf')
            plt.show()


    return list_temp, list_key_plot, list_full_data

def plot_corr_prion(datab, sol_dick, plot_list=[], threshold = 0.4, is_plot = False, func_from_key = None):

    list_temp = []
    list_bind_delta = []
    list_non_bind_delta = []
    ratio_bind_energy = []
    ratio_bind_temperature = []
    plotted_nb = 0
    non_plotted_nb = 0

    list_key_plot = []
    list_full_data = []

    for key in plot_list :

        plotted_nb+=1
        list_key_plot.append(key)

        path = "/Users/mathieuouellet/Desktop/AMP/AMP/src/polygon/test.feather"
        (x_t, p_t, dp_t), (r_12,r_21,dr_12,dr_21) = compute_markov(sol_dick, key, 0, path)
        (x_t_b, p_t_b, dp_t_b), (r_12_b,r_21_b,dr_12_b,dr_21_b) = compute_markov(sol_dick, key, 1, path)

        x_t, p_t, dp_t, r_12, r_21, dr_12, dr_21 = (list(t) for t in zip(*sorted(zip(x_t, p_t, dp_t, r_12, r_21, dr_12, dr_21))))
        x_t_b, p_t_b, dp_t_b, r_12_b, r_21_b, dr_12_b, dr_21_b = (list(t) for t in zip(*sorted(zip(x_t_b, p_t_b, dp_t_b, r_12_b, r_21_b, dr_12_b, dr_21_b))))


        list_full_data.append([(x_t, p_t, dp_t), (r_12,r_21,dr_12,dr_21),(x_t_b, p_t_b, dp_t_b), (r_12_b,r_21_b,dr_12_b,dr_21_b)])

        index_binded = next((i for i, x in enumerate(p_t_b) if x>threshold ), -1)
        index_non_binded = next((i for i, x in enumerate(p_t) if x>threshold ), -1)

        firt_t_binded = x_t_b[index_binded]
        firt_t_non_binded = x_t[index_non_binded]

        delta_binded = (x_t_b[index_binded+1] -  x_t_b[index_binded-1])/4
        delta_non_binded = (x_t[index_non_binded+1] -  x_t[index_non_binded-1])/4

        #print(*key,firt_t_non_binded,firt_t_binded, sep=',')
        list_temp.append((firt_t_non_binded,firt_t_binded,delta_non_binded,delta_binded ))

        binded_energy = datab.db_pairs[key[0]][key[1]]['sols'][key[2]]['NEB_HP_PP_energy']
        free_energy = datab.db_pairs[key[0]][key[1]]['sols'][key[2]]['NEB_HP_energy']
        delta_binded = max(binded_energy)-binded_energy[0]
        delta_free = max(free_energy)-free_energy[0]
        list_bind_delta.append(delta_binded)
        list_non_bind_delta.append(delta_free)
        ratio_bind_energy.append(delta_binded/delta_free)
        ratio_bind_temperature.append(firt_t_binded/firt_t_non_binded)
        print(key)
        print(datab.db_pairs[key[0]][key[1]]['sols'][key[2]]['conn'])
        print(func_from_key(key), list_temp[-1][1], list_temp[-1][0], list_temp[-1][1]-list_temp[-1][0] )

        if is_plot:

            fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
            #(x_t, p_t, dp_t)x_t_b, p_t_b, dp_t_b)
            ax[0,0].set_xscale('log')
            ax[0,0].errorbar(x_t_b, [1-p for p in p_t_b],   yerr=dp_t_b, fmt='X', c='g', label = "healthy (bind)")
            ax[0,0].errorbar(x_t_b, p_t_b,                   yerr=dp_t_b, fmt='X', c='r', label = "prion (bind)")
            #ax[0,0].plot(t_list, o_list_bind, 'o', c='y', label="other (bind)")
            ax[0,0].legend()

            ax[0,1].errorbar(x_t, [1-p for p in p_t], yerr=dp_t, fmt='x', c='g', label = "healthy (free)")
            ax[0,1].errorbar(x_t, p_t,                yerr=dp_t, fmt='x', c='r', label = "prion (free)")
            #ax[0,1].plot(t_list_bind, o_list, 'x', c='y', label="other (free)")
            ax[0,1].legend()
            ax[0,1].set_xscale('log')

            ax[1,0].set_xscale("log")
            ax[1,0].set_yscale('log')
            #cap for the plot for the infinite uncertainty one
            dr_12 = [ (dval if dval<1000*val else 0) for val,dval in zip(r_12, dr_12) ]
            dr_21 = [ (dval if dval<1000*val else 0) for val,dval in zip(r_21, dr_21) ]
            dr_12_b = [ (dval if dval<1000*val else 0) for val,dval in zip(r_12_b, dr_12_b) ]
            dr_21_b = [ (dval if dval<1000*val else 0) for val,dval in zip(r_21_b, dr_21_b) ]

            ax[1,0].errorbar(x_t, r_12,  yerr=dr_12,        fmt='x', c='r', label="r_12")
            ax[1,0].errorbar(x_t, r_21,  yerr=dr_21,        fmt='x', c='g', label="r_21")
            ax[1,0].errorbar(x_t_b, r_12_b,  yerr=dr_12_b,    fmt='X', c='r', label="r_12_b")
            ax[1,0].errorbar(x_t_b, r_21_b,  yerr=dr_21_b,    fmt='X', c='g', label="r_21_b")
            ax[1,0].legend()

            plot_binded(datab, key, ax[1,1])
            plt.savefig(f'./figure/test/rates_{key[0]}_{key[1]}_{key[2]}.pdf')
            plt.show()


    return list_temp, list_key_plot, list_full_data


In [ ]:
path = "/Users/mathieuouellet/Desktop/n=5_5deg_2_fig4b"
db_fig_4b = database.Database(path)
db_fig_4b.clear_all_lock()

with open('conf_of_dump_of_full_sim_2.dill', 'rb') as in_strm:
        dump_done_2, dump_dict_2 = dill.load(in_strm)
list_temp_2, list_key_plot_2, list_full_data_2 = plot_corr_prion(db_fig_4b, dump_dict_2, plot_list=[keys], threshold = 0.05, is_plot = True, func_from_key = lambda key: 1)



# Fig 3

In [ ]:
from scipy.odr import *


def funcArr(p, x):
            a,b,c, = p
            return a*np.exp( np.divide(- b, x-c) )


def get_beta(r12,r21,dr21,dr12, func, init):



    data_c = [[x[0],x[1],x[2],x[3]] for x in zip(r12,r21,dr21,dr12) if x[0]>0 or x[1]>0]


    r_12_c , r_21_c, dr_21_c, dr_12_c = list(zip(*data_c))
    # Create a model for fitting.
    quad_model = Model(func)
    # Create a RealData object using our initiated data from above.
    data_free = RealData( np.array(r_12_c),np.array(r_21_c), sy=np.array(dr_21_c), sx=(np.array(dr_12_c)))
    # Set up ODR with the model and data.
    odr = ODR(data_free, quad_model, beta0=init)
    # Run the regression.
    out = odr.run()


    return out.beta

def get_beta_nox(xt,r,dr, func, init, overwrite=False):



    data_c = [[x[0],x[1],x[2]] for x in zip(xt,r,dr) if x[0]>0 or x[1]>0]


    xt_c , r_c, dr_c = list(zip(*data_c))
    # Create a model for fitting.
    quad_model = Model(func)
    # Create a RealData object using our initiated data from above.
    data_free = RealData( np.array(xt_c),np.array(r_c), sy=np.array(dr_c))
    # Set up ODR with the model and data.
    odr = ODR(data_free, quad_model, beta0=init)
    # Run the regression.
    out = odr.run()


    return out.beta, func

In [ ]:
list_beta12 = []
list_beta12_b = []
list_beta21 = []
list_beta21_b = []

def plot_():

    for i in range(len(list_full_data_2)):
        (x_t, p_t, dp_t), (r_12,r_21,dr_12,dr_21),(x_t_b, p_t_b, dp_t_b), (r_12_b,r_21_b,dr_12_b,dr_21_b) = list_full_data_2[i]


        fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(10,10))


        def removeZero(x,r,dr):
            return list(zip( *[(v[0],v[1],v[2]) for v in zip(x,r,dr ) if v[1]!=0]))

        


        for axi in range(2):

            v = removeZero(x_t,r_12,dr_12)
            axs[0,axi].errorbar(np.divide(1,np.array(v[0])),v[1], marker='o', yerr=v[2], ls='none', label="free 12", c='b')
            v = removeZero(x_t_b,r_12_b,dr_12_b)
            axs[0,axi].errorbar(np.divide(1,np.array(v[0])),v[1], marker='o', label="bound 12",  yerr= v[2],  ls='none', c='y')
            v = removeZero(x_t,r_21,dr_21)
            axs[0,axi].errorbar(np.divide(1,np.array(v[0])),v[1],marker='o', yerr= v[2], ls='none', label="free 21", c='c')
            v = removeZero(x_t_b,r_21_b,dr_21_b)
            axs[0,axi].errorbar(np.divide(1,np.array(v[0])),v[1], marker='o', label="bound 21",  yerr= v[2],  ls='none', c='r')


        def clearHigherThan(x,r,dr, maxT, minT,index=0):
            return list(zip( *[(v[0],v[1],v[2]) for v in zip(x,r,dr ) if v[index]< maxT and v[index]> minT]))

        def clearZero(lst):
            return [1 for l in lst]


        dr_12 = clearZero(dr_12)
        dr_12_b = clearZero(dr_12_b)
        dr_21 = clearZero(dr_21)
        dr_21_b = clearZero(dr_21_b)

        min_T =  1/(10E4)
        max_T =1/(6E3)
        x_t_12,r_12,dr_12 = clearHigherThan(x_t,r_12,dr_12,max_T,min_T)
        x_t_21,r_21,dr_21 = clearHigherThan(x_t,r_21,dr_21,max_T,min_T)

        min_T =  1/(2E4)
        max_T =1/(4E3)
        x_t_b_21,r_21_b,dr_21_b = clearHigherThan(x_t_b,r_21_b,dr_21_b,max_T,min_T)

        min_T =  1/(2E4)
        max_T =1/(4E3)
        x_t_b_12,r_12_b,dr_12_b = clearHigherThan(x_t_b,r_12_b,dr_12_b,max_T,min_T)


        overwrite = False

        #plt.plot( [1E-6,1E-2],[1E-6,1E-2])
        func = funcArr
        init =  [ 3.16641779e-02,  4.93747260e-04, -7.42202232e-06]

        #[ 3E-1,   1E-5, 0]# ,-1.47747508]


        beta12, func12 = get_beta_nox(x_t_12,r_12,dr_12,func,init, overwrite )
        beta12_b, func12_b = get_beta_nox(x_t_b_12,r_12_b,dr_12_b,func,init, overwrite)
        beta21, func21 = get_beta_nox(x_t_21,r_21,dr_21,func,init,overwrite )
        beta21_b, func21_b = get_beta_nox(x_t_b_21,r_21_b,dr_21_b,func,init,overwrite)


        x_fit = np.linspace( math.log(1E-6), math.log(5E-3), 100)
        x_fit = np.exp(x_fit)
        inv_x_fit = np.divide(1,x_fit)

        y_fit_12 = func12(beta12, x_fit)
        y_fit_b_12 = func12_b(beta12_b, x_fit)
        y_fit_b_21 = func21_b(beta21_b, x_fit)
        y_fit_21 = func21(beta21, x_fit)

        for axi in range(2):
            if axi==0:
                linestyle='solid'
            else:
                linestyle='dashed'
            axs[0,axi].plot(inv_x_fit, y_fit_12, label ="free fit", c='b', linestyle=linestyle)
            axs[0,axi].plot(inv_x_fit, y_fit_b_12, label ="bound fit", c='y', linestyle=linestyle)
            axs[0,axi].plot(inv_x_fit, y_fit_21, label ="free fit", c='c', linestyle=linestyle)
            axs[0,axi].plot(inv_x_fit, y_fit_b_21, label ="bound fit", c='r', linestyle=linestyle)

        list_beta12.append([func12,beta12])
        list_beta12_b.append([func12_b, beta12_b])
        list_beta21.append([func21, beta21])
        list_beta21_b.append([func21_b, beta21_b])


        for axi in range(2):
            axs[0,axi].set_xlabel("rate P -> H")
            axs[0,axi].set_ylabel("rate H -> P")
            axs[0,axi].set_xscale('log')
            axs[0,axi].set_yscale('log')
            #ax.legend()

        axs[0,1].set_ylim(1E-3,1E-1)
        axs[0,1].set_xlim(1E2,7E3)

        axs[0,0].set_xlim(4E3,4E4)
        axs[0,0].set_ylim(1E-9,2E-2)

        

        #############################
        #      SET THE CURVE        #
        #############################

        list_prionicity = []

        tspace = np.exp(np.linspace(math.log(1E-6), math.log(1E-4),100))

        p = np.divide( np.clip(func12(beta12,tspace),0,1), np.clip(func12(beta12,tspace),0,1)+np.clip(func21(beta21,tspace),0,1)+1E-10)
        p_b = np.divide( np.clip(func12_b(beta12_b,tspace),0,1), np.clip(func12_b(beta12_b,tspace),0,1)+np.clip(func21_b(beta21_b,tspace),0,1)+1E-10)
        
        tp50= np.min(np.where(p>0.1,tspace,100000))
        tp50_b= np.min(np.where(p_b>0.1,tspace,100000))

        list_prionicity.append([(tp50-tp50_b)/(tp50_b+tp50),tp50,tp50_b])

        #axs[1,0].set_xlim((5E-6,30E-4))
        axs[1,0].plot(np.divide(1,tspace),p)
        axs[1,0].plot(np.divide(1,tspace),p_b)
        axs[1,0].set_xscale("log")
        axs[1,0].set_ylim(0,1.1)
        axs[1,0].set_xlim(2E2,2E5)

        #x_t, p_t, dp_t
        (x_t, p_t, dp_t), (r_12,r_21,dr_12,dr_21),(x_t_b, p_t_b, dp_t_b), (r_12_b,r_21_b,dr_12_b,dr_21_b) = list_full_data_2[i]

        x_t_c, p_t_c, dp_t_c= clearHigherThan(x_t, p_t, dp_t,1,1E-4, index=1)
        x_t_b_c, p_t_b_c, dp_t_b_c= clearHigherThan(x_t_b, p_t_b, dp_t_b,1,1E-4, index=1)

        #axs[1,0].plot(x_t, p_t)
        axs[1,0].errorbar(np.divide(1,x_t_c),p_t_c, marker='o', yerr= dp_t_c,  c='b', ls='none')
        axs[1,0].errorbar(np.divide(1,x_t_b_c), p_t_b_c, marker='o', yerr= dp_t_b_c,  c='r', ls='none')


        #############################
        #      SET THE CURVE        #
        #############################


        binded_rate = [ (abs(1/(r[0]-r[1])) if (r[0]-r[1])!=0 else -1) for r in  zip(r_21_b, r_12_b)]
        binded_rate_d = [(math.sqrt(r[0]**2 + r[1]**2)/((r[0]+r[1])**2) if (r[0]+r[1])>0 else -1) for r in  zip(dr_21_b, dr_12_b)]

        rate = [ (abs(1/(r[0]-r[1])) if (r[0]-r[1])!=0 else -1) for r in  zip(r_21, r_12)]
        rate_d = [(math.sqrt(r[0]**2 + r[1]**2)/((r[0]+r[1])**2)  if (r[0]+r[1])>0 else -1) for r in  zip(dr_21, dr_12)]

        def find_nearest(array, value):
            array = np.asarray(array)
            idx = (np.abs(array - value)).argmin()
            return idx



        index_nearest = [find_nearest(np.array(x_t_b),v) for v in x_t]
        ratio = [ (rate[i]/binded_rate[index_nearest[i]] if rate[i]!=0 else -1) for i in range(len(x_t)) ]

        

        axs[1,1].plot(np.divide(1,np.array(x_t)), ratio, 'o')
        axs[1,1].errorbar(np.divide(1,np.array(x_t_b)),binded_rate, marker='o', yerr= binded_rate_d,  c='r', ls='none')
        axs[1,1].errorbar(np.divide(1,np.array(x_t)),rate, marker='o', yerr= rate_d,  c='b', ls='none')
        axs[1,1].set_yscale("log")
        axs[1,1].set_xscale("log")

        axs[0,0].legend()
        axs[0,1].legend()

        plt.savefig(f'./figure/main_prion.pdf')

        plt.show()


plot_()

# Manifold plot (suppl 4)

In [ ]:

def get_min_energy_matrix(pca_data, energy, boundary, n_box ,min_E, max_E):

    x_list = np.linspace(boundary[0][0],boundary[0][1] ,n_box)
    y_list = np.linspace(boundary[1][0],boundary[1][1] ,n_box)

    min_E_log = math.log(min_E)
    max_E_log = math.log(max_E)

    Emin = np.full((n_box,n_box),max_E_log)
    count = np.zeros((n_box,n_box))

    for i in range(len(pca_data)):
        x,y = pca_data[i][0], pca_data[i][1]
        dx = x_list[1]-x_list[0]
        dy = y_list[1]-y_list[0]
        if x>=boundary[0][0] and x<=boundary[0][1] and y>=boundary[1][0] and y<=boundary[1][1]:
            indx = math.floor(abs((boundary[0][0]-x)/dx))
            indy = math.floor(abs((boundary[1][0]-y)/dy))
            #print(indx,indy)
            if energy[i] < Emin[indy,indx]:
                if energy[i] < min_E_log :
                    Emin[indy,indx] = min_E_log
                else:
                    Emin[indy,indx] = energy[i]

            count[indy,indx] = count[indy,indx] +  1
        
    return Emin, count, (dx,dy)

In [ ]:

is_plot_path = True

index_pca_binded = [0,4,2,3,1]

minx,maxx = -1.2,1.5
miny,maxy =  -0.3,0.3
min_E_exponent = -9
n_box = 100
max_E = 1.0

min_E = math.exp(min_E_exponent)
boundary = [[minx,maxx],[miny,maxy]]

#get the data point in the form of list of x1,y1,x2,y2
text_path = f"./pca_folder/{keys[0]}_{keys[1]}_{keys[2]}_low_temp.txt"
X = np.loadtxt(text_path, delimiter=',')
X_noerr = X[:,1:]
X_noerr_x = X_noerr[:,0:5]
X_noerr_y = X_noerr[:,7:12]
X_noerr = np.concatenate((X_noerr_x, X_noerr_y), axis=1)
energy  = np.log(X[:,0] + min_E)


output_path_list, vec_out = create_gif_transition_H_P(5, db, [keys], [PRE, PATH_DUMP, PATH_SCR, PATH_NEB, PATH_FINAL], full_output = True)


#get the pca
pca = PCA(n_components=2)
pca.fit(X_noerr)

#fit the data and get the grid for the plot
pca_data = pca.transform(X_noerr)
Emin, count, (dx,dy) = get_min_energy_matrix(pca_data, energy, boundary, n_box ,min_E, max_E)

#load binded data
text_path_bind = f"./pca_folder/{keys[0]}_{keys[1]}_{keys[2]}_binded_low_temp.txt"
Xbind = np.loadtxt(text_path_bind, delimiter=',')
Xbind_noerr = Xbind[:,1:]
Xbind_noerr_x = Xbind_noerr[:,0:5]
Xbind_noerr_y = Xbind_noerr[:,7:12]
Xbind_noerr = np.concatenate((Xbind_noerr_x, Xbind_noerr_y), axis=1)

#fit the data for binded and get the grid for plot
pca_data_bind = pca.transform(Xbind_noerr)
energy_bind = np.log(Xbind[:,0] + min_E)
Emin_bind, count_bind, (dx,dy) = get_min_energy_matrix(pca_data_bind, energy_bind, boundary, n_box ,min_E, max_E)

#plot both energy heatmap
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.scatter([v[0] for v in pca_data], [v[1] for v in pca_data],c=energy,  alpha = 0.1)
ax1.set_xlim(minx,maxx )
ax1.set_ylim(miny,maxy)
ax2.scatter([v[0] for v in pca_data_bind], [v[1] for v in pca_data_bind],c=energy_bind,  alpha = 0.1)
ax2.set_xlim(minx,maxx )
ax2.set_ylim(miny,maxy)
plt.show()


plt.set_cmap('viridis')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,7))
ax1.imshow(Emin, vmin=-7.5, vmax=-4.5)
im = ax2.imshow(Emin_bind, vmin=-7.5, vmax=-4.5)

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(im, cax=cbar_ax)

if is_plot_path:

    #set up path
    pos_list = get_neb_pos_list(vec_out[2])
    path_h_x = pos_list[:,0:5]
    path_h_y = pos_list[:,7:12]
    path_h = np.concatenate((path_h_x, path_h_y), axis=1)
    dim2_out_path_h= pca.transform(path_h)

    trans_func_test = get_translation_binded_function_neb(db, keys[0], keys[1], keys[2])
    list_pos_bottom, list_pos_top = trans_func_test(atom_list_list_binded)

    #set up path binded
    pos_list_bind = get_neb_pos_list(list_pos_bottom, zero_on = 0, rotate_on = 1)
    index_pca_binded=[0,1,2,3,4]
    path_bind_x = np.take(pos_list_bind,index_pca_binded,1)
    path_bind_y = np.take(pos_list_bind, [7+v for v in index_pca_binded],1)
    path_bind = np.concatenate((path_bind_x, path_bind_y), axis=1)
    dim2_out_path_bind= pca.transform(path_bind)

    ax1.plot([ abs((minx-v[0])/dx) for v in dim2_out_path_h] ,[ abs((miny-v[1])/dy) for v in dim2_out_path_h] , c='r')
    ax2.plot([ abs((minx-v[0])/dx) for v in dim2_out_path_bind] ,[ abs((miny-v[1])/dy) for v in dim2_out_path_bind] , c='r')



plt.savefig(f'./figure/energymap{keys[0]}_{keys[1]}_{keys[2]}_{sum(pca.explained_variance_ratio_)}.pdf')

plt.show()

# Trace plot

In [ ]:
todo = []
temp_todo = [] 
id_todo = [] 
temp_target = 0.000316227766016838

for path, currentDirectory, files in os.walk("/Volumes/Lab_math/sim_prion_2/dump"):
    for file in files:
        if file.startswith("dump__1737806584325749351_-8771402139612191995_-4281357692739835561_True_"):
            temp = file.split("_")[6]
            if abs(float(temp) - temp_target)/temp_target<0.05: 
                ind = file.split("_")[7]
                todo.append(path + "/" + file)
                temp_todo.append(temp)
                id_todo.append(ind)
    
all_signal = []
all_signal_type = []

nb_done = 0 
for ppp, temp_ppp, ind in zip(todo,temp_todo, id_todo):


    d = dump.dump(ppp)  


    nsnaps = len(d.snaps)
    natoms = d.snaps[0].natoms
    data_array_x = np.zeros((nsnaps,natoms))
    data_array_y = np.zeros((nsnaps,natoms))

    columns = []
    for name in ['x','y']:
        columns.append(d.names[name])

    id = d.names["id"]


    for i, snap in enumerate(d.snaps):
        atoms = snap.atoms
        for j in range(natoms):
            id_atom =  int(atoms[j][0])-1
            data_array_x[i,id_atom] = atoms[j][columns[0]]
            data_array_y[i,id_atom] = atoms[j][columns[1]]


    toadd = 0
    for kn in range(28):

        xlist = np.array([data_array_x[:,i-1].tolist() for i in range(1+kn*7,8+kn*7)]).T
        ylist = np.array([data_array_y[:,i-1].tolist() for i in range(1+kn*7,8+kn*7)]).T
        frame_pos_list = [[[data_array_x[j,i-1],data_array_y[j,i-1]] for j in range(data_array_x.shape[0])] for i in range(1+kn*7,8+kn*7)] 

       

        oneD_list = []
        if kn%2==0 and kn>0:
            toadd +=2.5

        for frame in range(len(frame_pos_list[0])):

            pos_frame = [frame_pos_list[i][frame] for i in range(len(frame_pos_list)) ] 

            pos_frame_x = [pos_frame[k][0]-pos_frame[0][0]  for k in range(len(pos_frame))]
            pos_frame_y = [pos_frame[k][1]-pos_frame[0][1]  for k in range(len(pos_frame))]

            theta = math.atan2(pos_frame_y[1], pos_frame_x[1])
            c, s = np.cos(-theta), np.sin(-theta)
            R = np.array(((c, -s), (s, c)))
            pos_array = [ np.dot(R,np.array([x,y])) for x,y in zip(pos_frame_x,pos_frame_y) ] 

            pos_frame_x = np.array([v[0] for v in pos_array ])
            pos_frame_y = np.array([v[1]  for v in pos_array ])

            val = pca.transform( np.concatenate((pos_frame_x[0:5],  pos_frame_y[0:5])).reshape(1, -1)  )
            oneD_list.append(val[0][0]+toadd)
        
        all_signal.append(oneD_list)

In [ ]:
i=0
plt.plot(all_signal[2*i], c='r')
plt.plot(all_signal[2*i+1], c='g')
plt.savefig(f'./figure/special_telegraph{5.000000000000001e-04}.pdf')
plt.show()

# Plot NEB energy (fig 2)

In [ ]:

def get_reaction_coordinate_function(database, keys, nb_int, nb_out, isplot = False):
    _NEB_HP_atom_list_list = database.db_pairs[keys[0]][keys[1]]['sols'][keys[2]]['NEB_HP_atom_list_list']
    _NEB_HP_PP_atom_list_list = database.db_pairs[keys[0]][keys[1]]['sols'][keys[2]]['NEB_HP_PP_atom_list_list']


    pos_list = get_neb_pos_list(_NEB_HP_atom_list_list)
    path_h_x = pos_list[:,0:nb_out]
    path_h_y = pos_list[:,nb_out+nb_int:2*nb_out+nb_int]
    path_h = np.concatenate((path_h_x, path_h_y), axis=1)
    #get the pca
    pca = PCA(n_components=2)
    pca.fit(path_h)
    #fit the data and get the grid for the plot

    def func_translate(to_translate): 


        
        pos_list = get_neb_pos_list(to_translate)
        path_h_x = pos_list[:,0:nb_out]
        path_h_y = pos_list[:,nb_out+nb_int:2*nb_out+nb_int]
        path_h = np.concatenate((path_h_x, path_h_y), axis=1)
        return pca.transform(path_h)

    pca_data = func_translate(_NEB_HP_atom_list_list)
    endpoint_pca = [pca_data[0], pca_data[-1]]

    trans_func_test = get_translation_binded_function_neb(database, keys[0], keys[1], keys[2])

    def func_translate_binded(to_translate):
        list_pos_bottom, list_pos_top = trans_func_test(to_translate)
        pos_list_bind = get_neb_pos_list(list_pos_bottom)
        index_pca_binded=list(range(0,nb_out))
        path_bind_x = np.take(pos_list_bind,index_pca_binded,1)
        path_bind_y = np.take(pos_list_bind, [nb_out+nb_int+v for v in index_pca_binded],1)
        path_bind = np.concatenate((path_bind_x, path_bind_y), axis=1)
        return pca.transform(path_bind)
        

    pca_data_bind = func_translate_binded(_NEB_HP_PP_atom_list_list)
    endpoint_pca_bind = [pca_data_bind[0], pca_data_bind[-1]]

    if isplot:
        plt.plot([v[0] for v in pca_data], [v[1] for v in pca_data],  'o')
        plt.plot([v[0] for v in pca_data_bind], [v[1] for v in pca_data_bind],  'o')
    return  (func_translate, func_translate_binded), (endpoint_pca, endpoint_pca_bind)


key_h, key_p, key_s = keys
binded_energy = db.db_pairs[key_h][key_p]['sols'][key_s]['NEB_HP_PP_energy']
plt.plot([i/(len(energy_list_binded)-1) for i in range(len(energy_list_binded))], energy_list_binded,'o', label="bound")
energy = db.db_pairs[key_h][key_p]['sols'][key_s]['NEB_HP_energy']
plt.plot([i/(len(vec_out[1])-1) for i in range(len(vec_out[1]))], vec_out[1],'o', label="separated")
plt.legend()
plt.savefig(f'./figure/reaction_path_{keys[0]}_{keys[1]}_{keys[2]}_{sum(pca.explained_variance_ratio_)}.pdf')
plt.show()